In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from pandas import DataFrame, Series
from __future__ import division # show flaoting number
import warnings
warnings.filterwarnings("ignore")

### Lable encoder

In [ ]:
from sklearn.preprocessing import Imputer 
df=DataFrame([[1, 2], [np.nan, 3], [7, 6]])
#    0   1
#0  1.0  2
#1  NaN  3
#2  7.0  6
imp = Imputer(missing_values='NaN', strategy='mean', axis=0) #strategy='mean'('median','most_frequent')
imp.fit(df)
imp.transform(df)
#    0   1
#0  1.0  2
#1    4  3
#2  7.0  6
#######################################################
df=DataFrame({'column1':[1,4,2,1], 'column2':[1,2,2,1]})
#    column1  column2
#0      1        1
#1      4        2    
#2      2        2  
#2      1        1     

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)df['column1'].values
poly.fit_transform(df.values) # create a new feature matrix of polynomial degree = 2 combinations 
#if a sample is [a, b], the degree-2 polynomial features are [1, a, b, a^2, ab, b^2].
#array([[  1.,   1.,   1.,   1.,   1.,   1.],
#       [  1.,   4.,   2.,  16.,   8.,   4.],
#       [  1.,   2.,   2.,   4.,   4.,   4.],
#       [  1.,   1.,   1.,   1.,   1.,   1.]])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.preprocessing import LabelEncoder#fit_transform(df['A'].values), inverse_transform(y)
le =LabelEncoder()
X_new=le.fit_transform(df['column1'].values) #array([0, 2, 1, 0], dtype=int64) 
le.classes_  #array([1, 2, 4], dtype=int64)
le.inverse_transform(X_new) #array([1, 4, 2, 1], dtype=int64)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
train=pd.DataFrame({'column1':['A', 'C', 'B']})
test=pd.DataFrame({'column1':['A', 'B', 'D']})
df_obj = train.select_dtypes(include=['object'])
for i in df_obj: #i shows each 'object' column name
    total=list(train[i].values.astype('str')) + list(test[i].values.astype('str')) #['A', 'C', 'B', 'A', 'B', 'D']
    encoder=LabelEncoder()
    encoder.fit(total)
    train[i]=encoder.transform(list(train[i].values.astype('str')))
    test[i] =encoder.transform(list(test[i].values.astype('str')))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from pandas import factorize
factorize(df['column1'])[0] #array([0, 1, 2, 0], dtype=int64)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df=DataFrame({'column1':['A','B','A'], 'column2':['C','C','D']})
df_obj = df.select_dtypes(include=['object'])
for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]
#df_obj
#      column1  column2
#0        0        0
#1        1        0
#2        0        1
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categorical_features=[0])#[0] : only encode the first column'; 'all' : encode all columns
enc.fit_transform(df).toarray() 
#array([[ 1.,  0.,  0.,  1.],
#       [ 0.,  0.,  1.,  2.],
#       [ 0.,  1.,  0.,  2.],
#       [ 1.,  0.,  0.,  1.]])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.preprocessing import Binarizer
B=Binarizer(threshold=1) # if a value > threshold, set to 1. Otherwise, set to 0. 
B.fit_transform(df['column1'].values) #array([[0, 1, 1, 0]], dtype=int64)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer(neg_label=0, pos_label=1)
lb.fit_transform(df['column1'])
#array([[1, 0, 0],
#       [0, 0, 1],
#       [0, 1, 0],
#       [1, 0, 0]])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
df=pd.DataFrame({'column1':['A', 'B', 'C']})
df = pd.concat([df, pd.get_dummies(df['column1'], prefix='column1')],axis=1)
df.drop(['column1'],axis=1, inplace=True)  
df
#     column1_A   column1_B   column1_C
#0        1           0          0
#1        0           1          0
#2        0           0          1
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
df=pd.DataFrame({'column1':['A', 'B', 'A'],'column2':['C', 'D', 'C'],
                 'column3':[2.0, 3.0, 1.0],'column4':[4.0, 3.0, 2.0] })
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
features = []
for f in df.select_dtypes(include=['object']):
    #print f
    dummy = pd.get_dummies(df[f].astype('category'))
    dummy = csr_matrix(dummy)#
    #print dummy.toarray()
    features.append(dummy)

scaler = StandardScaler()
float_columns_scaled=scaler.fit_transform(df[df.select_dtypes(exclude=['object']).columns])
float_columns_scaled = csr_matrix(float_columns_scaled)
features.append(float_columns_scaled)
features=hstack(features, format = 'csr').toarray()
#array([[ 1.        ,  0.        ,  1.        ,  0.        ,  0.        ,  1.22474487],
#       [ 0.        ,  1.        ,  0.        ,  1.        ,  1.22474487,    0.      ],
#       [ 1.        ,  0.        ,  1.        ,  0.        , -1.22474487, -1.22474487]])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import theano
from keras.utils.np_utils import to_categorical #like onehotencoder
to_categorical(df['column1'].values,5) #number of classes =5
#array([[ 0.,  1.,  0.,  0.,  0.],
#       [ 0.,  0.,  0.,  0.,  1.],
#       [ 0.,  0.,  1.,  0.,  0.],
#       [ 0.,  1.,  0.,  0.,  0.]]
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
train=pd.DataFrame({'column1':[1, 3, 4]})
test=pd.DataFrame({'column1':[1, 2, 3]})
categories=list(train['column1'].unique()) + [f for f in test['column1'].unique() if f not in train['column1'].unique()]
train=pd.concat([train,DataFrame(to_categorical(train['column1'], len(categories)+1))],axis=1)
#     column1    0    1     2    3     4
#0      1       0.0  1.0   0.0  0.0   0.0
#1      3       0.0  0.0   0.0  1.0   0.0
#2      2       0.0  0.0   0.0  0.0   1.0
test=pd.concat([test,DataFrame(to_categorical(test['column1'], len(categories)+1))],axis=1)
#     column1    0    1     2    3     4
#0      1       0.0  1.0   0.0  0.0   0.0
#1      3       0.0  0.0   1.0  0.0   0.0
#2      2       0.0  0.0   0.0  1.0   0.0
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.feature_extraction import DictVectorizer
df = pd.DataFrame({'A' : ['a', 'b', 'a','c'], 'B' : ['e', 'x', 'x','e']})
cols=['A','B']
vec = DictVectorizer()
mkdict = lambda row: dict((col, row[col]) for col in cols)
X=df[cols].apply(mkdict, axis=1)
#0    {u'A': u'a', u'B': u'e'}
#1    {u'A': u'b', u'B': u'x'}
#2    {u'A': u'a', u'B': u'x'}
#3    {u'A': u'c', u'B': u'e'
vec.fit_transform(X).toarray()
#array([[ 1.,  0.,  0.,  1.,  0.],
#       [ 0.,  1.,  0.,  0.,  1.],
#       [ 1.,  0.,  0.,  0.,  1.],
#       [ 0.,  0.,  1.,  1.,  0.]])
vecData = pd.DataFrame(vec.fit_transform(X).toarray())
vecData.columns = vec.get_feature_names()
vecData.index = df.index
df.join(vecData).drop(cols, axis=1)
#     A=a   A=b   A=c   B=e    B=x
#0    1.0   0.0   0.0   1.0    0.0
#1    0.0   1.0   0.0   0.0    1.0
#2    1.0   0.0   0.0   0.0    1.0
#3    0.0   0.0   1.0   1.0    0.0

### Feature selection and extraction

In [ ]:
# Regression
X = [[10, 0, 5], 
     [20, 1, 10], 
     [30, 0, 12], 
     [35, 1, 20]]
Y = [1,2,3,4]
from sklearn.feature_selection import VarianceThreshold#remove some features with low variance

np.var(X, axis=0) #array([ 92.1875,   0.25  ,  29.1875])
sel = VarianceThreshold(threshold=1)
sel.fit_transform(X)
#array([[10,  5],
#       [20, 10],
#       [30, 12],
#       [35, 20]])
#############################################################################################
from sklearn.feature_selection import f_regression #each linear model fit one of the features
fvalue, pvalue = f_regression(X, Y)
zip(fvalue, pvalue) ##choose pvalue <=0.05, for those features are highly correlated with the target Y
#[(96.333333333332732, 0.010221733442710707),
# (0.49999999999999994, 0.55278640450004202),  # delete this feature
# (35.063492063492014, 0.027354872578278844)]

from sklearn.feature_selection import SelectKBest 
selector = SelectKBest(f_regression, k=2) #k : Number of top features to select
selector.fit_transform(X, Y)
#array([[10,  5],
#       [20, 10],
#       [30, 12],
#       [35, 20]])
zip(selector.scores_ ,selector.pvalues_) 
#[(96.333333333332732, 0.010221733442710707),
# (0.49999999999999994, 0.55278640450004202),
# (35.063492063492014, 0.027354872578278844)]
#############################################################################################
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X, Y)
model = SelectFromModel(clf, threshold=0.1, prefit=True)
#Features whose importance is greater or equal are kept while the others are discarded.
X_new = model.transform(X)
#array([[ 0,  5],
#       [ 1, 10],
#       [ 0, 12],
#       [ 1, 20]])
model.get_support() #array([False,  True,  True], dtype=bool)
#############################################################################################
from sklearn.feature_selection import RFE #recursive feature elimination
from sklearn.linear_model import LinearRegression
lr = LinearRegression()#use lr to calculate each feature's weight
#rank all features, i.e continue the elimination until the last ones
rfe = RFE(lr, n_features_to_select=2)
X_new = rfe.fit_transform(X,Y)
#array([[ 0,  5],
#       [ 1, 10],
#       [ 0, 12],
#       [ 1, 20]])
rfe.ranking_ #array([2, 1, 1]) #1 if selected
rfe.support_ #array([False,  True,  True], dtype=bool) (show each feature is selected or not (True or False))

In [ ]:
# Classification
Y = [1,0,0,1]
X = [[10,0, 5], 
     [2, 1, 10], 
     [3, 1, 12], 
     [10,0, 20]]
from sklearn.feature_selection import f_classif #Compute the ANOVA F-value for the provided sample
fvalue, pvalue = f_classif(X, Y)
zip(fvalue, pvalue) ##choose pvalue <=0.05, for those features are highly correlated with the target Y
#[(225.0, 0.0044150325), (inf, 0.0), (0.03930131, 0.86117655)]
#############################################################################################
from sklearn.feature_selection import chi2 #Compute chi-squared stats between each non-negative feature and class
chi2statistics, pvalue = chi2(X, Y)
zip(chi2statistics, pvalue) #choose pvalue <=0.05, for those features are highly correlated with the target Y
#[(9.0, 0.0026997960632601883),
# (2.0, 0.15729920705028505),
# (0.19148936170212766, 0.66167991486599997)]
#############################################################################################
from sklearn.feature_selection import SelectKBest 
selector = SelectKBest(f_classif, k=2) #k : Number of top features to select
selector.fit_transform(X, Y)
#array([[10,  0],
#       [ 2,  1],
#       [ 3,  1],
#       [10,  0]])
zip(selector.scores_ ,selector.pvalues_) #[(225.0, 0.0044150325), (inf, 0.0), (0.03930131, 0.86117655)]
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
selector = SelectKBest(chi2, k=2) #k : Number of top features to select
X_new=selector.fit_transform(X, Y)
zip(selector.scores_ ,selector.pvalues_)
#[(9.0, 0.0026997960632601883),
# (2.0, 0.15729920705028505),
# (0.19148936170212766, 0.66167991486599997)]
#############################################################################################
from sklearn.feature_selection import SelectPercentile
selector = SelectPercentile(f_classif, percentile=60) 
#selector = SelectPercentile(chi2, percentile=60) 
selector.fit_transform(X, Y)
#array([[10,  0],
#       [ 2,  1],
#       [ 3,  1],
#       [10,  0]])
selector.get_support() #array([ True,  True, False], dtype=bool) show each feature is selected or not 
#############################################################################################
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X, Y)
model = SelectFromModel(clf, threshold=0.3, prefit=True)
#Features whose importance is greater or equal are kept while the others are discarded.
X_new = model.transform(X)
X_new
#array([[10,  0],
#       [ 2,  1],
#       [ 3,  1],
#       [10,  0]])
model.get_support() #array([ True,  True, False], dtype=bool)
#############################################################################################
from sklearn.feature_selection import mutual_info_classif
mutual_info_classif(X,Y) #Return estimated mutual information between each feature and the target. 
#array([ 0.83333333,  0.33333333,  0.08333333])
# MI value=0 if two variables are independent

from sklearn.feature_selection import SelectKBest 
selector = SelectKBest(mutual_info_classif, k=2) #k : Number of top features to select
selector.fit_transform(X, Y)
#array([[10,  0],
#       [ 2,  1],
#       [ 3,  1],
#       [10,  0]])

In [ ]:
Y = [1,0,0,1]
X = [[10, 1, 0], 
     [-1, 0, 1], 
     [-1, 0, 1], 
     [10, 1, 0]]
from sklearn.decomposition import DictionaryLearning 
#Finds a dictionary (a set of atoms) that can best be used to represent data using a sparse code
dic = DictionaryLearning(n_components=2, max_iter=500) #n_components : number of classes
dic.fit_transform(X)
#array([[-10.04987562,   0.        ],
#       [  0.        ,   1.41421356],
#       [  0.        ,   1.41421356],
#       [-10.04987562,   0.        ]])
#for each sample, all values=0 except a class column which it belongs to
#############################################################################################
Y = [0,1,1,2,2]
X =[[  5.1,  3.5,  1.4,  0.2],
    [ 7. ,  3.2,  4.7,  1.4],
    [ 6.4,  3.2,  4.5,  1.5],
    [ 6.2,  3.4,  5.4,  2.3],
    [ 5.9,  3. ,  5.1,  1.8]]
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=2) 
#Number of components (< n_classes - 1) for dimensionality reduction.
lda.fit(X, Y).transform(X)
#array([[-7.92849875,  0.03078069],
#       [ 2.70626992, -1.73129046],
#       [ 1.12104434, -0.51183983],
#       [ 2.66031756,  1.89878759],
#       [ 1.44086692,  0.31356201]])
lda.explained_variance_ratio_  #array([ 0.94059285,  0.05940715])
#Percentage of variance explained by each of the selected components
#############################################################################################
X = [[10, 0, 5], 
     [20, 1, 10], 
     [30, 0, 12], 
     [35, 1, 20]]
from sklearn.decomposition import FactorAnalysis 
fa = FactorAnalysis(n_components=2)
fa.fit_transform(X)
#array([[ 1.39947678, -0.06007879],
#       [ 0.36108626, -0.63879181],
#       [-0.47340936,  1.49222757],
#       [-1.28715367, -0.79335696]])
#############################################################################################
from sklearn.decomposition import TruncatedSVD#truncated singular value decomposition#can operate on sparse x data
svd = TruncatedSVD(n_components=2, n_iter=10, random_state=42)
svd.fit(X) 
X_reduced=svd.transform(X) 
X_reduced
#array([[ 11.17829772,   0.05080662],
#       [ 22.37567529,  -0.07911717],
#       [ 32.19050963,   2.78538999],
#       [ 40.26344149,  -2.19704902]])
#############################################################################################
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit_transform(X)
pca.fit(X)
pca.transform(X)
#array([[ 15.31173002,  -0.57962934],
#       [  4.11954044,  -0.35523   ],
#       [ -5.60110411,   2.82751203],
#       [-13.83016635,  -1.89265269]])
pca.explained_variance_ratio_ #array([ 0.97444102,  0.02474639])
plt.semilogy(pca.explained_variance_ratio_, '--o')
plt.semilogy(pca.explained_variance_ratio_.cumsum(), '--o')
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
fig, ax=plt.subplots(nrows=2, ncols=1, figsize=(8, 12))
ax[0].semilogy(pca.explained_variance_ratio_, '--o',label='explained variance ratio');
ax[0].set_title('explained variance ratio')
ax[0].set_ylim([10**-7, 1])
ax[1].plot(pca.explained_variance_ratio_.cumsum(), '--o', label='cumulative explained variance ratio');
ax[1].set_title('cumulative explained variance ratio')
ax[1].set_xlabel('Principle components ')
ax[1].set_ylim([0.2, 1])
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(12,5), facecolor='white')
ax.plot(pca_trafo.explained_variance_ratio_.cumsum(),'--o',linewidth=3)
ax.hlines(y=0.99, xmin=0, xmax=14, lw=2,color='r')
ax.set_xlabel('Number of PCA components')
ax.set_ylabel('Cumulative explained variance')
ax.grid(color='black', linestyle='--', linewidth=1)
#############################################################################################
from sklearn.decomposition import KernelPCA
pca = KernelPCA(kernel="rbf", n_components=2, gamma=1)
#kernel=“linear” (default)| “poly” | “rbf” | “sigmoid” | “cosine” 
pca.fit_transform(X)
pca.fit(X)
pca.transform(X)
#array([[ 0.        ,  0.        ],
#       [-0.57735027, -0.57735027],
#       [-0.21132487,  0.78867513],
#       [ 0.78867513, -0.21132487]])
#############################################################################################
from sklearn.manifold import MDS #multidimentional scaling 
mds = MDS(n_components=2)
mds.fit_transform(X)
#array([[ -4.91628672,  14.46430834],
#       [ -2.49385278,   3.46170037],
#       [ -0.45664302,  -6.50418256],
#       [  7.86678252, -11.42182615]])

### Scaler

In [63]:
#if the regularization is applied, all features need to be scaled
#Decision trees and random forests are one of the very few machine learning algorithms that don't need to scale features 
X = [[1, 2], [3, 4], [5, 6], [7, 8]]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaler.transform(X)
scaler.fit_transform(X)
#array([[-1.34164079, -1.34164079],
#       [-0.4472136 , -0.4472136 ],
#       [ 0.4472136 ,  0.4472136 ],
#       [ 1.34164079,  1.34164079]])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.preprocessing import scale 
scale(X, axis=0, with_mean=True, with_std=True)#enter=0, std=1
#array([[-1.34164079, -1.34164079],
#       [-0.4472136 , -0.4472136 ],
#       [ 0.4472136 ,  0.4472136 ],
#       [ 1.34164079,  1.34164079]])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X)
scaler.transform(X)
scaler.fit_transform(X)
#array([[ 0.        ,  0.        ],
#       [ 0.33333333,  0.33333333],
#       [ 0.66666667,  0.66666667],
#       [ 1.        ,  1.        ]])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.preprocessing import normalize
normalize(X) #scale each samle to a length of 1
#array([[ 0.4472136 ,  0.89442719],
#       [ 0.6       ,  0.8       ],
#       [ 0.6401844 ,  0.76822128],
#       [ 0.65850461,  0.75257669]])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Normalize each feature column by the median of the feature values
df=DataFrame(X)
df.apply(lambda x: x / np.median(x), axis=0) 
#     0      1
#0   0.25   0.4
#1   0.75   0.8
#2   1.25   1.2
#3   1.75   1.6

### Text preprocess

In [ ]:
import nltk
nltk.download() #various corpus will be saved in your PC
from nltk.stem.wordnet import WordNetLemmatizer 
W = WordNetLemmatizer()
W.lemmatize('dogs') #u'dog'
W.lemmatize('gathering','v') # u'gather'
W.lemmatize('gathering','n') #'gathering'
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from nltk.stem.porter import PorterStemmer 
words = ["gaming","gamed","games"]
ps = PorterStemmer()
[ps.stem(i) for i in words] #[u'game', u'game', u'game']
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
rom nltk.stem import SnowballStemmer 
words = ["gaming","gamed","games"]
s = SnowballStemmer('english')
[s.stem(i) for i in words] #[u'game', u'game', u'game']
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from nltk.stem import LancasterStemmer 
words = ["gaming","gamed","games"]
lan = LancasterStemmer()
[lan.stem(i) for i in words] #['gam', 'gam', 'gam']
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from nltk.corpus import stopwords
words = ["how","are","dogs"]
[w for w in words if not w in stopwords.words('english')]  #['dogs']
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from nltk.tokenize import word_tokenize#A=word_tokenize('a book')
sentence = 'Please buy me a book.\n\nThanks.'
s=word_tokenize(sentence) #['Please', 'buy', 'me', 'a', 'book', '.', 'Thanks', '.']

from nltk import FreqDist
FreqDist(s) #show the appearing times for each token
#FreqDist({'.': 2,
#          'Please': 1,
#          'Thanks': 1,
#          'a': 1,
#          'book': 1,
#          'buy': 1,
#          'me': 1})
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from nltk import pos_tag, word_tokenize
pos_tag(word_tokenize('a book'))
#[('a',DT), ('book',NN)]       #NN:noun, JJ:adjective, DT:determiner, VB:verbs, RB:adverbs
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from nltk import bigrams
b=list(bigrams('This is book'))
#[('T', 'h'),
# ('h', 'i'),
# ('i', 's'),
# ('s', ' '),
# (' ', 'i'),
# ('i', 's'),
# ('s', ' '),
# (' ', 'b'),
# ('b', 'o'),
# ('o', 'o'),
# ('o', 'k')]
from collections import Counter
Counter(b)
#Counter({(' ', 'b'): 1,
#         (' ', 'i'): 1,
#         ('T', 'h'): 1,
#         ('b', 'o'): 1,
#         ('h', 'i'): 1,
#         ('i', 's'): 2,
#         ('o', 'k'): 1,
#         ('o', 'o'): 1,
#         ('s', ' '): 2})
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
rom string import punctuation
from string import lower
from string import split
text= 'This is a book, and a car.'
exclude= ["is", "a"]
' '.join(x for x in (word.strip(punctuation) for word in text.split()) if x.lower() not in exclude) #'This book and car'

text.split() #['This', 'is', 'a', 'book,', 'and', 'a', 'car.']
[word.strip(punctuation) for word in text.split()] #['This', 'is', 'a', 'book', 'and', 'a', 'car']
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from string import punctuation
from string import lower
from string import split
text= 'This is a book, and a car.'
exclude= ["is", "a"]
text.split() #['This', 'is', 'a', 'book,', 'and', 'a', 'car.']
[word.strip(punctuation) for word in text.split()] #['This', 'is', 'a', 'book', 'and', 'a', 'car']

' '.join(x for x in (word.strip(punctuation) for word in text.split()) if x.lower() not in exclude) #'This book and car'
' '.join(x for x in (word.strip(punctuation) for word in text.split())) #'This is a book and a car'

from string import maketrans
from string import translate 
text.translate(maketrans("",""), punctuation) #'This is a book and a car'
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
text= 'This is a! book@, and a car.'
from re import sub
print re.split('\W+', text) 
#['This', 'is', 'a', 'book', 'and', 'a', 'car', '']
print sub('[\w]+',' ', text.lower()) 
#     !  @,      .    .
#re.sub with a special pattern as the first argument. Matches are replaced with an empty string (removed). 

v = """<p id=1>Sometimes, <b>simpler</b> is better,but <i>not</i> always.</p>"""
# Replace HTML tags with an empty string.
re.sub("<.*?>", "", v)#'Sometimes, simpler is better,but not always.'
#<      Less-than sign (matches HTML bracket).
#.*?    Match zero or more chars.
#>      Greater-than (matches HTML bracket).

text = "He was carefully disguised but captured quickly by police."
re.findall(r"\w+ly", text) #['carefully', 'quickly']
' '.join("a in_book")                         #'a   i n _ b o o k'
' '.join(["a", "in_book"])                    #'a in_book'
' '.join(["a", "in_book"]).replace('_', ' ')  #'a in book'

### Text feature extraction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
texts=["a dog cat fish","the dog cat cat","any fish bird"]
cv = CountVectorizer(stop_words='english',lowercase=True)
#index items have no stop words, Convert all characters to lowercase before tokenizing.
X_train_counts = cv.fit_transform(texts)
cv.fit_transform(texts).toarray()
cv.fit_transform(texts).todense()
#array([[0, 1, 1, 1],
#       [0, 2, 1, 0],
#       [1, 0, 0, 1]], dtype=int64)
cv.get_feature_names()  #[u'bird', u'cat', u'dog', u'fish']
cv.vocabulary_ #{u'bird': 0, u'cat': 1, u'dog': 2, u'fish': 3}

from sklearn.feature_extraction.text import TfidfTransformer
tf=TfidfTransformer(norm="l2",smooth_idf=True)
tf.fit_transform(X_train_counts).toarray()
#array([[ 0.        ,  0.57735027,  0.57735027,  0.57735027],
#       [ 0.        ,  0.89442719,  0.4472136 ,  0.        ],
#       [ 0.79596054,  0.        ,  0.        ,  0.60534851]])
##################################################################################################
from sklearn.feature_extraction.text import TfidfVectorizer #(CountVectorizer  + TfidfTransformer)
texts=["a dog cat fish","the dog cat cat","any fish bird"]
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit_transform(texts).toarray()
#array([[ 0.        ,  0.57735027,  0.57735027,  0.57735027],
#       [ 0.        ,  0.89442719,  0.4472136 ,  0.        ],
#       [ 0.79596054,  0.        ,  0.        ,  0.60534851]])
texts=["a dog cat fish","the dog cat cat","any fish bird"]
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit_transform(texts).toarray()
#array([[ 0.        ,  0.57735027,  0.57735027,  0.57735027],
#       [ 0.        ,  0.89442719,  0.4472136 ,  0.        ],
#       [ 0.79596054,  0.        ,  0.        ,  0.60534851]])
vectorizer.get_feature_names() #u'bird', u'cat', u'dog', u'fish']
#~~~~~~~~~~~~~~~~~~~~~~~~~~~
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,3)) #ngram_range=(min words, max words)
vectorizer.fit_transform(texts).toarray()
#array([[ 0.        ,  0.36617957,  0.        ,  0.48148213,  0.36617957,
#         0.36617957,  0.        ,  0.48148213,  0.36617957,  0.        ],
#       [ 0.        ,  0.6503311 ,  0.42755362,  0.        ,  0.32516555,
#         0.32516555,  0.42755362,  0.        ,  0.        ,  0.        ],
#       [ 0.62276601,  0.        ,  0.        ,  0.        ,  0.        ,
#         0.        ,  0.        ,  0.        ,  0.4736296 ,  0.62276601]])
vectorizer.get_feature_names()
#[u'bird',
# u'cat',
# u'cat cat',
# u'cat fish',
# u'dog',
# u'dog cat',
# u'dog cat cat',
# u'dog cat fish',
# u'fish',
# u'fish bird']
#~~~~~~~~~~~~~~~~~~~~~~~~~~~
texts=["a dog cat fish :d","the dog cat cat \br\b","any fish bird"]
def P1(text):
    text=text.lower()
    emoticons={':d':'good',':)':'good',':(':'good',}
    E=[k for k in list(emoticons.keys())]
    for i in E:
        text=text.replace(i, emoticons[i])
    abbreviations={r"\br\b": "are", r"\bu\b": "you"}
    for i, j in abbreviations.items():
        text=re.sub(i,j, text)#replace i with j# ex: i=\br\b, j=are
    return text
vectorizer = TfidfVectorizer(preprocessor=P1 )
vectorizer.fit_transform(texts).toarray()
vectorizer.get_feature_names() #[u'any', u'are', u'bird', u'cat', u'dog', u'fish', u'good', u'the']
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.feature_extraction.text import HashingVectorizer 
#turns a collection of text documents into a sparse matrix of token occurrence counts, no IDF weighting 
texts=["a dog cat fish :d","the dog cat cat \br\b","any fish bird"]
hv = HashingVectorizer(n_features=2**2,ngram_range=(1,1),stop_words='english',lowercase =True,preprocessor=P1)
hv.fit_transform(texts).toarray()
#array([[ 0.        ,  0.        ,  0.        ,  1.        ],
#       [ 0.        , -0.4472136 ,  0.        ,  0.89442719],
#       [ 0.        , -0.70710678,  0.        ,  0.70710678]])

### Image preprocess

In [ ]:
from mahotas import imread
imread('test.jpg') # show each pixel with RGB values # shape is (4128L, 3096L, 3L)
img=imread('test.jpg',as_grey=True) # shape is (4128L, 3096L) ##each pixel value is 0~255 float values

from mahotas import imsave
mh.imsave('copy.png', img)

from mahotas import stretch
im_stretch=stretch(img, 0,100) #stretch the image to the range of min:0 and max:100

from mahotas.colors import rgb2grey #Compute luminance of an RGB image
im=imread('test.jpg')
rgb2grey(im, dtype=np.uint8) # shape is (4128L, 3096L)  #each pixel value is 0~255 int values

from mahotas import gaussian_filter #can remove very noisy edges
image=imread('test.jpg',as_grey=True) #shape is (4128L, 3096L)
image_filter=gaussian_filter(image, sigma=16) #sigma : standard deviation for Gaussian kernel
plt.imshow(image_filter)
#####################################################################################
from skimage.color import rgb2hsv 
#RGB to HSV (Hue, saturation, Value/Brightness) color space conversion
im=imread('test.jpg')
rgb2hsv(im) # shape is (4128L, 3096L, 3L)
#####################################################################################
from skimage.measure import find_contours 
x, y = np.ogrid[-np.pi:np.pi:100j, -np.pi:np.pi:100j]
image = np.sin(np.exp((np.sin(x)**3 + np.cos(y)**2)))

# Find contours at a constant value of 0.8
contours =find_contours(r, 0.8)

# Display the image and plot all contours found
fig, ax = plt.subplots()
ax.imshow(r,interpolation='nearest', cmap=plt.cm.gray)
# plot a list of found contours at a constant value of 0.8
for n, contour in enumerate(contours):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)
# plot the outer edge
contour_max=max(contours, key=len)
ax.plot(contour_max[:, 1], contour_max[:, 0], linewidth=10)
ax.set_xticks([])
ax.axis('image')
ax.set_yticks([])
plt.show()
#####################################################################################
from scipy.misc import imread
im=imread('test.jpg') # show each pixel with RGB values # shape is (4128L, 3096L, 3L)
plt.imshow(im)
from scipy.ndimage import center_of_mass #Calculate the center of mass of an image
center_of_mass(im) #(17409.064992314547, 12272.500769857432, 7.4503332729033245

x = np.array([[0,0,0,0],
              [0,1,1,0],
              [0,1,1,0],
              [0,1,1,0],
              [0,0,0,0]])
center_of_mass(x) #(2.0, 1.5) #Coordinates of centers-of-mass
#####################################################################################
import cv2
from cv2 import imread, imwrite
from cv2 import imshow
from cv2 import waitKey
from cv2 import destroyAllWindows
from cv2 import resize
from cv2 import INTER_AREA, INTER_LINEAR

im=imread('test.jpg',1) # show each pixel with RGB values # shape is (4128L, 3096L, 3L) 
cv2.imwrite('copy.png',im)# save an image
imshow('image',im)
waitKey(0)
destroyAllWindows()

im=imread('test.jpg',0) # show each pixel with 0~255 int values # shape is (4128L, 3096L)
height, width = im.shape
im_resize = resize(im, (int(width/10), int(height/10)), interpolation = INTER_AREA)
#interpolation = INTER_LINEAR (default)
#INTER_AREA - resampling using pixel area relation. It is similar to the INTER_NEAREST method.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
if im.shape[0] > im.shape[1]:
    scaled_size = (256, int(im.shape[0]*256/im.shape[1]))
else:
    scaled_size = (int(im.shape[1]*256/im.shape[0]),256)
im_resize = cv2.resize(im, dsize=scaled_size) 
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
plt.imshow(im_resize)
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

# Convert color to HSV
hsvimage = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
# Convert color to gray
grayimage= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#check of an image is blury or not
blurness=cv2.Laplacian(grayimage, cv2.CV_64F).var()

#convert a color image to a bi-level image by using an optimal threshold. 
a=2
thresh = (grayimage.max())/a
maxValue = 255
_, outputimage = cv2.threshold(grayimage, thresh, maxValue, cv2.THRESH_BINARY);
#the outputimage show 0 or maxValue. 
#The gray value of grayimage above thresh set to the value of maxValue (ex: 255), less than thresh set to 0
contours, hierarchy = cv2.findContours(outputimage,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
main_contour = sorted(contours, key = cv2.contourArea, reverse = True)[0] #choose the outest contour
# set the blank image
ff = np.zeros((gray.shape[0],gray.shape[1]), 'uint8') 
# Draw the outest contour in the blank image
cv2.drawContours(ff, main_contour,-1, 1, 20) #(destination_image, contours,contourIdx, contour_color, contour_thickness)
#if contours is negative, all the contours are drawn.
ff_mask = np.zeros((gray.shape[0]+2,gray.shape[1]+2), 'uint8')
# fill red color inside the outest contour
cv2.floodFill(ff, ff_mask, (int(gray.shape[1]/2), int(gray.shape[0]/2)),1)
#(image, mask, start_pont, newVal) # Floodfill started from point (int(gray.shape[1]/2), int(gray.shape[0]/2))
# newVal– New value of the repainted domain pixels.
plt.imshow(ff)
#####################################################################################
from PIL import Image
im = Image.open("test.jpg") # show a color image
#im = Image.open("test.jpg").convert('L') # show a gray image
im=im.resize((100,100),Image.ANTIALIAS) #ANTIALIAS is a high-quality downsampling filter
im=im.crop((0,0,80,80))# define the left, upper, right, and lower pixel coordinate
#(0, 0) in the upper left corner
im1=np.asarray(im.getdata())#Returns pixel values

### Image feature extraction

In [ ]:
from mahotas import imread
from mahotas.features import surf 
image=imread('test.jpg',as_grey=True)
features=surf.surf(image, descriptor_only=True) #shape is (1013L, 64L), s has (num of interesting points, 64 descriptors)

s=surf.surf(image, 4, 6, 1)##draws coloured polygons around the interest points
f=surf.show_surf(image, s)
plt.imshow(f)

surf.dense(image, spacing=64) #Descriptors at dense points in a distance of 64 pixels from each other
# shape is (2684L, 64L)
#####################################################################################
from mahotas import imread
from mahotas.features import lbp #local binary pattern
image=imread('test.jpg',as_grey=True)
features=lbp(image, radius=30, points=12)# shape is (352L,), histograms of the pixel code counts
#####################################################################################
from mahotas import imread
from mahotas.features import haralick
#calculate the probability that a pixel with the gray level i is adjacent to a pixel with the gray level j 
image=imread('test.jpg',as_grey=True)
features=haralick(image.astype('int32')).ravel() #shape is (52L,) 4 directions and 14 statistics for each direction
#####################################################################################
from mahotas import imread
from skimage.filters import sobel# perform edge detection using the Sobel transform
image=imread('test.jpg',as_grey=True) #shape is (4128L, 3096L)
features=sobel(image)  #shape is (4128L, 3096L)
#####################################################################################
#color histogram features (put pixel color values into histogram) 
from mahotas import imread
image=imread('test.jpg')
image = image // 64
r, g, b=image.transpose((2,0,1))# each pixel value of r (or g or b) is 0,1,2 or 3# each image has 4*4*4=64 different colors
pixelvalue=r+4*b+16*g
H=np.bincount(pixelvalue.ravel(), minlength=64).astype(float) #shape : (64L,)
features=np.log1p(H)#shape : (64L,)

from mahotas import imread
from cv2 import calcHist
image=imread('test.jpg')
hist=calcHist([image], [1,2,0], None, [4, 4, 4], [0, 256, 0, 256, 0, 256]) #shape : (4,4,4)
hist=hist.ravel() #shape : (64L,)
features= np.log1p(hist)#shape : (64L,)
#####################################################################################
image=np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
from skimage.feature import corner_harris, corner_peak
image_corner=corner_peaks(corner_harris(image))# show the coordinates of the corners
#array([[2, 2],
#       [2, 7],
#       [7, 2],
#       [7, 7]], dtype=int64
plt.imshow(image, cmap='Set3')
plt.scatter(image_corner[:,1],image_corner[:,0]) #show corner marks
#####################################################################################
from skimage.transform import downscale_local_mean 
image=np.array([[ 1,2,0,1],
            [ 1,0,1,8],
            [ 1,0,1,1],
            [0,0,2,2]])
downscale_local_mean(image, (2, 2))
#array([[ 1.  ,  2.5 ],
#       [ 0.25,  1.5 ]])
#####################################################################################
from scipy.ndimage import binary_erosion, binary_dilation, binary_opening, binary_closing
image=np.array([[0, 0, 0, 0, 0],
                [0, 1, 1, 1, 0],
                [0, 1, 1, 1, 0],
                [0, 1, 1, 1, 0],
                [0, 0, 0, 0, 0]])
binary_erosion(image).astype(a.dtype)# shrinking the shapes in an image
#array([[0, 0, 0, 0, 0],
#       [0, 0, 0, 0, 0],
#       [0, 0, 1, 0, 0],
#       [0, 0, 0, 0, 0],
#       [0, 0, 0, 0, 0]])
binary_dilation(image).astype(a.dtype)# expanding the shapes in an image.
#array([[0, 1, 1, 1, 0],
#       [1, 1, 1, 1, 1],
#       [1, 1, 1, 1, 1],
#       [1, 1, 1, 1, 1],
#       [0, 1, 1, 1, 0]])
binary_opening(image).astype(a.dtype)#the succession of an erosion and a dilation 
#array([[0, 0, 0, 0, 0],
#       [0, 0, 1, 0, 0],
#       [0, 1, 1, 1, 0],
#       [0, 0, 1, 0, 0],
#       [0, 0, 0, 0, 0]])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
image=np.array([[0, 0, 0, 0, 0],
                [0, 1, 1, 1, 0],
                [0, 1, 0, 1, 0],
                [0, 1, 1, 1, 0],
                [0, 0, 0, 0, 0]])

binary_closing(image).astype(a.dtype)#the succession of a dilation and an erosion 
#array([[0, 0, 0, 0, 0],
#       [0, 1, 1, 1, 0],
#       [0, 1, 1, 1, 0],
#       [0, 1, 1, 1, 0],
#       [0, 0, 0, 0, 0]])
#####################################################################################

### Split dataset, pipeline, cross validation and params search


In [165]:
Y = [1,0,0,1]
X = [[10,0, 5], 
     [2, 1, 10], 
     [3, 1, 12], 
     [10,0, 20]]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

############################ Pipeline   ##################################################
from sklearn.decomposition import PCA
from sklearn.svm import SVC
pca = PCA(random_state=0,n_components=2)
classifier = SVC(random_state=0,C=1)

#~~~~~~~~~~~~~
from sklearn.pipeline import Pipeline
pipe = Pipeline(steps=[('pca', pca), ('clf', classifier)])
#~~~~~~~~~~~~~
from sklearn.pipeline import make_pipeline 
pipe = make_pipeline(pca, classifier)
#~~~~~~~~~~~~~

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
pipe.score(X_train, y_train)
pipe.named_steps['pca'].n_components #2 , show the value of pca_n_components

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X = [[10,0, 5], 
     [2, 1, 10], 
     [3, 1, 12], 
     [10,0, 20]]
from sklearn.pipeline import FeatureUnion
pca = PCA(random_state=0,n_components=3)
Kernelpca = KernelPCA(kernel="rbf", n_components=2, gamma=1)
combined = FeatureUnion([('linear_pca', pca), ('kernel_pca',Kernelpca)])
combined.fit_transform(X)
#array([[ -5.87353048e+00,   5.03752193e+00,   3.53522640e-03, 5.00841535e-01,   7.07106781e-01],
#       [ -2.55786711e+00,  -3.85080151e+00,  -6.03620876e-02, -5.00841535e-01,  -5.06429716e-14],
#       [ -4.01757915e-01,  -3.27243849e+00,   6.88251651e-02, -5.00841535e-01,  -3.40261343e-15],
#       [  8.83315551e+00,   2.08571806e+00,  -1.19983039e-02,  5.00841535e-01,  -7.07106781e-01]])

###########################   ParameterGrid ###############################################
from sklearn.model_selection import ParameterGrid
param_grid = {'a': [1, 2], 'b': [True, False]}
list(ParameterGrid(param_grid))
#[{'a': 1, 'b': True},
# {'a': 1, 'b': False},
# {'a': 2, 'b': True},
# {'a': 2, 'b': False}]

##############################  GridSearchCV  ##############################################
from sklearn.pipeline import Pipeline
pca = PCA(random_state=0)
n_components = range(1,3,1)
classifier = SVC(random_state=0)
C = np.logspace(-3,3,7)
pipe = Pipeline(steps=[('pca', pca), ('clf', classifier)])

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,dict(pca__n_components=n_components, clf__C=C), scoring='accuracy', n_jobs=-1,cv=3)
#scoring='log_loss'

from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
my_scorer=make_scorer(score_func=f1_score, greater_is_better=True)
grid = GridSearchCV(pipe,dict(pca__n_components=n_components, clf__C=C), scoring=my_scorer, n_jobs=-1,cv=3)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.model_selection import RandomizedSearchCV
grid = RandomizedSearchCV(pipe,param_distributions=dict(pca__n_components=n_components, clf__C=C), scoring='accuracy', 
                          n_jobs=-1,cv=3,n_iter=4)
# randomly pick up n_iter parameter combibation, For example: n_iter=4
#{'pca__n_components': 1, 'clf__C': 1000}
#{'pca__n_components': 1, 'clf__C': 100}
#{'pca__n_components': 3, 'clf__C': 100}
#{'pca__n_components': 2, 'clf__C': 0.001}
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

grid.fit(X_train, y_train)
for params, mean_score, scores in grid.grid_scores_: 
    print '%.5f +/- %.5f %r' %(mean_score, scores.std(),params)
print 'Best parameters set found on development set:%r' % (grid.best_params_) 
print 'Best average accuracy in validation set: %.5f' %(grid.best_score_)
print 'Accuracy in test set: %.5f' %(grid.score(X_test,y_test))
y_pred=grid.predict(X_test)
############################## manually grid search ###############################
best_score, best_param = 0, None
for p1 in [1,5,3]:
    params=(p1)
    try:
        accuracy = ......
        if accuracy > best_score:
            best_score, best_param = accuracy, params
    except:
        continue
print('Best parameters =%s accuracy=%.3f' % (best_param, best_score)) 
######################   cross validation  #########################################
X = np.array([[1, 10], 
              [2, 20], 
              [3, 30], 
              [4, 40], 
              [5, 50], 
              [6, 60]])
y = np.array([1,1,1,0,0,0])
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, random_state=0, shuffle=True)#shuffle the data before splitting into batches
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#('TRAIN:', array([0, 1, 3, 4], dtype=int64), 'TEST:', array([2, 5], dtype=int64))
#('TRAIN:', array([0, 2, 4, 5], dtype=int64), 'TEST:', array([1, 3], dtype=int64))
#('TRAIN:', array([1, 2, 3, 5], dtype=int64), 'TEST:', array([0, 4], dtype=int64))
#************************************************************
kfolds=KFold(n_splits=10, shuffle=True,random_state=0)
trainindex=[]
validindex=[]
for train_index, valid_index in kf.split(X):
    trainindex.append(train_index)
    validindex.append(valid_index)
trainindex=np.array(trainindex)
validindex=np.array(validindex)
###########################
foldnumber=0 #1, 2....9
X_train, X_valid = X[trainindex[foldnumber]], X[validindex[foldnumber]]
y_train, y_valid = y[trainindex[foldnumber]], y[validindex[foldnumber]]          
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.model_selection import StratifiedKFold#preserving the percentage of samples for each class.
kf = StratifiedKFold(n_splits=3, random_state=0, shuffle=True)#shuffle the data before splitting into batches
for train_index, test_index in kf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#('TRAIN:', array([0, 1, 3, 4], dtype=int64), 'TEST:', array([2, 5], dtype=int64))
#('TRAIN:', array([1, 2, 3, 5], dtype=int64), 'TEST:', array([0, 4], dtype=int64))
#('TRAIN:', array([0, 2, 4, 5], dtype=int64), 'TEST:', array([1, 3], dtype=int64))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#('TRAIN:', array([1, 2, 3, 4, 5], dtype=int64), 'TEST:', array([0], dtype=int64))
#('TRAIN:', array([0, 2, 3, 4, 5], dtype=int64), 'TEST:', array([1], dtype=int64))
#('TRAIN:', array([0, 1, 3, 4, 5], dtype=int64), 'TEST:', array([2], dtype=int64))
#('TRAIN:', array([0, 1, 2, 4, 5], dtype=int64), 'TEST:', array([3], dtype=int64))
#('TRAIN:', array([0, 1, 2, 3, 5], dtype=int64), 'TEST:', array([4], dtype=int64))
#('TRAIN:', array([0, 1, 2, 3, 4], dtype=int64), 'TEST:', array([5], dtype=int64))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.model_selection import ShuffleSplit
rs = ShuffleSplit(n_splits=3, test_size=.33333, random_state=0)
for train_index, test_index in rs.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#('TRAIN:', array([1, 3, 0, 4], dtype=int64), 'TEST:', array([5, 2], dtype=int64))
#('TRAIN:', array([4, 0, 2, 5], dtype=int64), 'TEST:', array([1, 3], dtype=int64))
#('TRAIN:', array([1, 2, 4, 0], dtype=int64), 'TEST:', array([3, 5], dtype=int64))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.3333, random_state=0)
for train_index, test_index in sss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#('TRAIN:', array([5, 2, 4, 0], dtype=int64), 'TEST:', array([3, 1], dtype=int64))
#('TRAIN:', array([3, 1, 0, 4], dtype=int64), 'TEST:', array([5, 2], dtype=int64))
#('TRAIN:', array([3, 1, 0, 4], dtype=int64), 'TEST:', array([5, 2], dtype=int64))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.model_selection import cross_val_score
clf = SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=3,scoring='accuracy',n_jobs=1)
print scores #[ 0.5  1.   1. ]
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#Accuracy: 0.83 (+/- 0.47)

#~~~~~~~~~~~~~~~~~~~~~~ walk-forward validation  ~~~~~~~~~~~~~~~~~~~~
#the training set is divided into the train set and validation set. 
#Each model is evaluated using walk-forward validation. 
#I use the last 50% of training set as the validation set.
#The validation set is iterated to test and evaluate each model. 
#In the beginning, the first 50% of training set is used as the train set to train a model. 
#For each iteration in the validation set: 
#(1) A model is trained by the train set. 
#(2) A prediction made by the model is performed on the i-th data in the validation set. 
#A one-step prediction for one data will be stored for later evaluation. 
#(3) The feature variables and the actual target value from the i-th data in the validation set will be 
#added to the train set for the next iteration. 
#In the end, the predictions made during the iteration of the validation set will be evaluated in terms of classification accuracy. 
for c in [0.01,0.1,1]:
    strain_size = int(len(x_train) * 0.50)
    xtrain, xvalidation = x_train[0:-train_size], x_train[-train_size:]
    ytrain, yvalidation = y_train.values[0:-train_size], y_train.values[-train_size:]
    xhistory=xtrain.copy()
    yhistory=ytrain.copy()
    predictions = []
    end=int(len(x_train) * 0.50)
    for i in range(0,end,1): 
        estimator = SVC(C=1, random_state=0)
        estimator.fit(xhistory, yhistory)       
        y_pred=estimator.predict(xvalidation[i])
        predictions.append(y_pred)
        xhistory=np.vstack((xhistory,xvalidation[i]))
        yhistory=np.r_[yhistory, yvalidation[i]]
    accuracy=accuracy_score(y_true=yvalidation, y_pred=predictions) 
    print '{}: {}'.format('Accuracy ', accuracy) 
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    
window=20 # 20 days

train_size = (int(len(x_train) * 0.50)/window)*window
xtrain, xvalidation = x_train[0:-train_size], x_train[-train_size:]
train, yvalidation = y_train.values[0:-train_size], y_train.values[-train_size:]
xhistory=xtrain.copy()
yhistory=ytrain.copy()
predictions = []
end=((int(len(x_train) * 0.50)/window)-1)*window+1
for i in range(0,end,window): 
    estimator = SVR(C=1, epsilon=epsi)
    estimator.fit(xhistory, yhistory)       
    y_pred=estimator.predict(xvalidation[i:window+i])
    predictions.append(y_pred)
    xhistory=np.vstack((xhistory,xvalidation[i:window+i]))
    yhistory=np.r_[yhistory, yvalidation[i:window+i]]
    #calculate the metrics for each validation fold
    submae=mean_absolute_error(y_true=yvalidation[i:window+i], y_pred=y_pred)

y_pred=np.ravel(predictions)
mae=mean_absolute_error(y_true=yvalidation, y_pred=y_pred)

### Regression and classification models

In [ ]:
from sklearn.linear_model import Perceptron#(n_iter, eta0, random_state)
estimator = Perceptron(n_iter=100, random_state=0, eta0=0.002)
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) #array([1, 0, 0])
estimator.coef_ #array([[ 0.016, -0.01 , -0.028]]) Weights assigned to the features
estimator.intercept_ #array([-0.01]) Constants in decision function
accuracy=estimator.score(X_train, y_train) #0.66666666666666663
########################################################################################
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression(C=1.0,penalty='l2', random_state=0)#penalty : 'l1' or 'l2'
estimator.fit(X_train, y_train)
y_prob=estimator.predict_proba(X_test)
#array([[ 0.24547008,  0.75452992],
#       [ 0.64501769,  0.35498231],
#       [ 0.51242289,  0.48757711]])
y_pred=estimator.predict(X_test) #array([1, 0, 0])
estimator.coef_ #Weights assigned to the features
estimator.intercept_ # Constants in decision function
accuracy=estimator.score(X_train, y_train)
########################################################################################
from sklearn.linear_model import SGDClassifier
estimator = SGDClassifier(penalty='l2', random_state=0,loss='log',l1_ratio=0.5,alpha=0.001)
#penalty : 'l1', 'l2', or 'elasticnet' #alpha: Constant that multiplies the regularization term
estimator.fit(X_train, y_train)
estimator.partial_fit(X_train, y_train, classes=np.array([0,1])) #Fit linear model with Stochastic Gradient Descent.
y_prob=estimator.predict_proba(X_test)
y_pred=estimator.predict(X_test) 
y_prob=estimator.predict_proba(X_test)
estimator.coef_ #Weights assigned to the features
estimator.intercept_ # Constants in decision function
accuracy=estimator.score(X_train, y_train)
########################################################################################
from sklearn.svm import SVC #support vector classification
estimator = SVC(kernel='rbf', gamma=0.1,C=10, probability=True, random_state=0)
estimator = SVC(kernel='linear', C=10, probability=True, random_state=0)
estimator = SVC(kernel='poly', gamma=0.1,degree=3,C=10, probability=True, random_state=0)
#C: Penalty parameter C of the error term
#gamma: Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.svm import NuSVC #Nu-Support Vector Classification
estimator = NuSVC(nu=0.5, kernel='rbf', gamma=0.1, probability=True, random_state=0)
estimator = NuSVC(nu=0.5, kernel='linear', probability=True, random_state=0)
estimator = NuSVC(nu=0.5, kernel='poly', gamma=0.1,degree=3, probability=True, random_state=0)
#gamma: Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
#nu:(0, 1], An upper bound on the fraction of training errors and a lower bound of the fraction of support vectors. 

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train, y_train)
y_prob=estimator.predict_proba(X_test)
y_pred=estimator.predict(X_test) 
#estimator.coef_ #Weights assigned to the features #coef_ is only available when using a linear kernel
estimator.intercept_ # Constants in decision function
accuracy=estimator.score(X_train, y_train)
########################################################################################
from sklearn.svm import LinearSVC#Linear Support Vector Classification.
estimator = LinearSVC(penalty='l2', C=1.0, random_state=0) 
#C: Penalty parameter C of the error term # penalty : 'l1', 'l2'
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
estimator.coef_ #Weights assigned to the features 
estimator.intercept_ # Constants in decision function
accuracy=estimator.score(X_train, y_train)
########################################################################################
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis #All the classes have the same covariance matrix
estimator = LinearDiscriminantAnalysis()
estimator.fit(X_train, y_train)
estimator.coef_ #Weights assigned to the features 
estimator.intercept_ # Constants in decision function

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#All the classes don't the same covariance matrix, fit better than LDA
estimator = QuadraticDiscriminantAnalysis()
estimator.fit(X_train, y_train)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
y_prob=estimator.predict_proba(X_test)
y_pred=estimator.predict(X_test) 
accuracy=estimator.score(X_train, y_train)
estimator.classes_ #array([0, 1]) Unique class labels
########################################################################################
from sklearn.naive_bayes import GaussianNB #Gaussian Naive Bayes 
#suppose the probability of a feature belong to a class has a normal distribution
estimator = GaussianNB()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.naive_bayes import BernoulliNB #for each feature has a binary value
estimator = BernoulliNB(alpha =1.0)
#alpha : smoothing parameter-->laplace:1, Lidstone:0.01,0.05,0.1,0.5, nonmmothing:0

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.naive_bayes import MultinomialNB #for text classification
#ex:TfidfVectorizer + MultinomialNB
estimator = MultinomialNB(alpha =1.0)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train, y_train)
y_prob=estimator.predict_proba(X_test)
y_pred=estimator.predict(X_test) 
accuracy=estimator.score(X_train, y_train)
estimator.classes_ #array([0, 1]) Unique class labels
########################################################################################
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
estimator = GaussianProcessClassifier(kernel=1.0 * RBF(length_scale=1.0) , n_restarts_optimizer=10, n_jobs =-1, random_state=0)
#n_restarts_optimizer:The number of restarts of the optimizer for finding the kernel’s parameters which maximize 
#the log-marginal likelihood.
estimator.fit(X_train, y_train)
y_prob=estimator.predict_proba(X_test)
y_pred=estimator.predict(X_test) 
accuracy=estimator.score(X_train, y_train)
estimator.classes_

########################################################################################
from sklearn.neighbors import KNeighborsClassifier
estimator = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski', n_jobs=-1)
estimator.fit(X_train, y_train)
y_prob=estimator.predict_proba(X_test)
y_pred=estimator.predict(X_test) 
accuracy=estimator.score(X_train, y_train)
estimator.classes_

########################################################################################
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
clf1 = LogisticRegression(random_state=1)
clf2 = GaussianNB()
estimator = VotingClassifier(estimators=[('lr', clf1), ('gnb', clf2)], voting='hard')
estimator = VotingClassifier(estimators=[('lr', clf1), ('gnb', clf2)], voting='soft', weights=[2,1])
#voting='hard', uses predicted class labels for majority rule voting. 
#voting='soft', predicts the class label based on the argmax of the sums of the predicted probabilities
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
accuracy=estimator.score(X_train, y_train)
########################################################################################
X_train=np.array([[ 5.1,  3.5,  1.4,  0.2],
            [ 4.9,  3. ,  1.4,  0.2],
            [ 4.7,  3.2,  1.3,  0.2],
            [ 5.4,  3.7,  1.5,  0.2],
            [ 6.4,  3.2,  4.5,  1.5],
            [ 6.9,  3.1,  4.9,  1.5]])
y_train=np.array([ 0,  0,  0, -1, 1, 1])
from sklearn.semi_supervised import LabelPropagation #Label Propagation classifier
estimator = LabelPropagation(kernel='rbf',gamma=20)
estimator = LabelPropagation(kernel='knn',n_neighbors=3)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.semi_supervised import LabelSpreading #has regulation
estimator = LabelSpreading(kernel='rbf',gamma=20)
estimator = LabelSpreading(kernel='knn',n_neighbors=3)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train, y_train)
y_prob=estimator.predict_proba(X_train)
y_pred=estimator.predict(X_train) #array([0, 0, 0, 0, 1, 1])
accuracy=estimator.score(X_train, y_train)
estimator.classes_ #array([0, 1]) Unique class labels
########################################################################################
## plot decision region
from mlxtend.plotting import plot_decision_regions
from sklearn import datasets
from sklearn.svm import SVC
# Loading some example data
iris = datasets.load_iris()
X = iris.data[:, [0, 2]]
y = iris.target
# Training a classifier
svm = SVC(C=0.5, kernel='linear')
svm.fit(X, y)

# Plotting decision regions
plot_decision_regions(X, y, clf=svm, res=0.02, legend=2)
# Adding axes annotations
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.title('SVM on Iris')
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
estimator = LinearRegression(n_jobs=-1)
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
estimator.coef_ #Weights assigned to the features 
estimator.intercept_ # Constants in decision function
########################################################################################
from sklearn.linear_model import SGDRegressor
estimator = SGDRegressor(penalty='l2', random_state=0,loss='squared_loss',l1_ratio=0.5,alpha=0.001)
#penalty : 'l1', 'l2', or 'elasticnet' #alpha: Constant that multiplies the regularization term
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared=estimator.score(X_test, y_test)
estimator.coef_ #Weights assigned to the features
estimator.intercept_ # Constants in decision function
########################################################################################
from sklearn.linear_model import RANSACRegressor #RANSAC (RANdom SAmple Consensus) algorithm.
from sklearn.linear_model import LinearRegression
base_estimator = LinearRegression()
estimator = RANSACRegressor(base_estimator, min_samples=2,residual_threshold=2, random_state=0,
                                   max_trials=100,loss='absolute_loss')
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
estimator.inlier_mask_ #array([ True,  True,  True,  True,  True,  True,  True], dtype=bool) # inliers classified as True.
estimator.estimator_.coef_ #Best fitted model Weights assigned to the features  
estimator.estimator_.intercept_
########################################################################################
from sklearn.linear_model import Ridge #Linear least squares with l2 regularization
estimator = Ridge(alpha=1.0, random_state=0) #alpha: Regularization strength

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.linear_model import Lasso #Linear Model trained with L1 prior as regularizer
estimator = Lasso(alpha=0.1, random_state=0) #alpha: Regularization strength

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.linear_model import ElasticNet #Linear regression with combined L1 and L2 priors as regularizer
estimator = ElasticNet(alpha=1, l1_ratio=0.5, random_state=0) #alpha: Regularization strength

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
estimator.coef_ #Weights assigned to the features 
estimator.intercept_ # Constants in decision function

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Lasso Path : plot various features' weights along the alpha values
from sklearn.linear_model import lasso_path
alphas_lasso, coefs_lasso, _ = lasso_path(X_train, y_train, eps = 5e-3)
#eps = Length of the path. eps=5e-3 means that alpha_min / alpha_max = 5e-3
neg_log_alphas_lasso = -np.log10(alphas_lasso)
for coef_l in coefs_lasso:
    plt.plot(neg_log_alphas_lasso, coef_l)
plt.xlabel('-Log(alpha)')
plt.ylabel('coefficients')
plt.axis('tight')
########################################################################################
from sklearn.linear_model import RidgeCV #Ridge linear model with iterative fitting along a regularization path
estimator = RidgeCV(alphas=(0.1, 1.0, 10.0), store_cv_values=True, cv=None) 
#cv =None Leave-One-Out cross-validation, cv=integer, to specify the number of folds.

estimator.cv_values_ #Cross-validation values for each alpha 
estimator.cv_values_.mean(axis=0).argmin() #show the best alpha index
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.linear_model import LassoCV #Lasso linear model with iterative fitting along a regularization path 
estimator = LassoCV(alphas=(0.1, 1.0, 10.0), cv=None, n_jobs=-1,random_state=0) 
#cv =None Leave-One-Out cross-validation, cv=integer, to specify the number of folds.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# for feature number > sample number,  ElasticNetCV can fit better than LinearRegression
from sklearn.linear_model import ElasticNetCV #Elastic Net model with iterative fitting along a regularization path
estimator = ElasticNetCV(alphas=(0.1, 1.0, 10.0), l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=None, n_jobs=-1, random_state=0) 

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
estimator.coef_ #Weights assigned to the features 
estimator.intercept_ # Constants in decision function
estimator.alpha_  #Estimated regularization parameter.
########################################################################################
from sklearn.linear_model import BayesianRidge #Bayesian ridge regression
estimator = BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06) 
#lambda_1, lambda_2 are the shape parameters and inverse scale parameter for feature wight#alpha_1, alpha_2 are for noise
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
estimator.coef_ #Weights assigned to the features 
estimator.lambda_ # estimated precision of the weights.
estimator.alpha_  #estimated precision of the noise
########################################################################################
from sklearn.linear_model import Lars #Least Angle Regression model # well suitable for feature number > sanple number
estimator = Lars(n_nonzero_coefs=2) #n_nonzero_coefs :the number of non_zero weights for features

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.linear_model import LarsCV #Cross-validated Least Angle Regression model
estimator = LarsCV(cv=None, n_jobs=-1)
#cv: None, the default 3-fold cross-validation / cv:integer, to specify the number of folds.
estimator.alpha_ #the estimated regularization parameter alpha

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
estimator.coef_ #Weights assigned to the features 
estimator.intercept_ # Constants in decision function
########################################################################################
from sklearn.svm import SVR #Support Vector Regression
estimator = SVR(kernel='rbf', gamma=0.1,C=10,epsilon=0.2)
estimator = SVR(kernel='linear', C=10,epsilon=0.2)
estimator = SVR(kernel='poly', gamma=0.1,degree=3,C=10,epsilon=0.2)
#C: Penalty parameter C of the error term
#gamma: Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
#epsilon: It specifies the epsilon-tube within which no penalty is associated in the training loss function 
#with points predicted within a distance epsilon from the actual value.

estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
estimator.coef_ #Weights assigned to the features  #coef_ is only available when using a linear kernel
estimator.intercept_ # Constants in decision function
########################################################################################
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
estimator = GaussianProcessRegressor(kernel=1.0 * RBF(length_scale=1.0), alpha =1e-2 , n_restarts_optimizer=10, random_state=0)
estimator = GaussianProcessRegressor(alpha =1e-2 , n_restarts_optimizer=10, random_state=0)
#alpha : Value added to the diagonal of the kernel matrix during fitting. 
#Larger values correspond to increased noise level in the observations
#n_restarts_optimizer:The number of restarts of the optimizer for finding the kernel’s parameters which maximize 
#the log-marginal likelihood.
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
########################################################################################
from sklearn.neighbors import KNeighborsRegressor
estimator = KNeighborsRegressor(n_neighbors=5, p=2, metric='minkowski', n_jobs=-1)
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
########################################################################################
## plot residual plot
plt.scatter(y_pred, y_pred-y_test)
plt.hlines(y=0, xmin=X_test.min(), xmax=X_test.max(), lw=2)

In [ ]:
#can support multi-class classification without one versus all techniques, # can be used for nonlinear data
from sklearn.tree import DecisionTreeClassifier
estimator = DecisionTreeClassifier(criterion='entropy',max_depth=10,min_samples_split=2, 
                                   min_samples_leaf=1, max_features='auto', random_state=0)
#max_features : The number of features to consider when looking for the best 
#If int, then consider max_features features at each split.
#If float, then max_features is a percentage and int(max_features * n_features) features are considered at each split.
#If “auto”, then max_features=sqrt(n_features).
#If “sqrt”, then max_features=sqrt(n_features).
#If “log2”, then max_features=log2(n_features).
#If None, then max_features=n_features.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier(n_estimators=10, criterion='entropy',max_depth=10,min_samples_split=2, 
                                   min_samples_leaf=1, max_features='auto',random_state=0, n_jobs=-1)


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import ExtraTreesClassifier
#randomly select some features and threshold for each split, grow all dataset
estimator = ExtraTreesClassifier(criterion='entropy',max_depth=10,min_samples_split=2, 
                                   min_samples_leaf=1, max_features='auto',random_state=0)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
base_estimator = DecisionTreeClassifier(criterion='entropy',max_depth=None, random_state=0)
estimator = BaggingClassifier(base_estimator=base_estimator, n_estimators=100, max_samples=0.8, max_features=0.8, 
                              bootstrap=True, bootstrap_features=False, n_jobs=-1, random_state=0)
#max_samples : int, then draw max_samples samples; float, then draw max_samples * X.shape[0] samples.
#max_features : int, then draw max_features features; float, then draw max_features * X.shape[1] features.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
base_estimator = DecisionTreeClassifier(criterion='entropy',max_depth=1, random_state=0)
estimator = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=10, learning_rate=0.1, random_state=0)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import GradientBoostingClassifier
estimator = GradientBoostingClassifier(n_estimators=10, max_depth=3, subsample=1.0, min_samples_split=2, 
                                       min_samples_leaf=1,learning_rate=0.1, max_features='auto', random_state=0)
#subsample : The fraction of samples to be used for fitting the individual base learners. 
#max_features : int, float, 'auto', 'sqrt', 'log2', None

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train, y_train)
y_prob=estimator.predict_proba(X_test)
y_pred=estimator.predict(X_test) 
accuracy=estimator.score(X_train, y_train)
estimator.classes_
estimator.feature_importances_ #The higher, the more important the feature #BaggingClassifier doesn't have
########################################################################################
########################################################################################
from sklearn.tree import DecisionTreeRegressor
estimator = DecisionTreeRegressor(criterion='mse',max_depth=10,min_samples_split=2, 
                                  min_samples_leaf=1, max_features='auto', random_state=0)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import RandomForestRegressor
estimator = RandomForestRegressor(n_estimators=10, criterion='mse',max_depth=10,min_samples_split=2, 
                                  min_samples_leaf=1, max_features='auto', random_state=0, n_jobs=-1)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import ExtraTreesRegressor
#randomly select some features and threshold for each split, grow all dataset
estimator = ExtraTreesRegressor(criterion='mse',max_depth=10,min_samples_split=2, 
                                  min_samples_leaf=1, max_features='auto', random_state=0)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
base_estimator = DecisionTreeRegressor(criterion='mse',max_depth=None, random_state=0)
estimator = BaggingRegressor(base_estimator=base_estimator, n_estimators=100, max_samples=0.8, max_features=0.8, 
                              bootstrap=True, bootstrap_features=False, n_jobs=-1, random_state=0)
#max_samples : int, then draw max_samples samples; float, then draw max_samples * X.shape[0] samples.
#max_features : int, then draw max_features features; float, then draw max_features * X.shape[1] features.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
base_estimator = DecisionTreeRegressor(criterion='mse',max_depth=1, random_state=0)
estimator = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=10, learning_rate=0.1, random_state=0)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.ensemble import GradientBoostingRegressor
estimator = GradientBoostingRegressor(n_estimators=10, max_depth=3, subsample=1.0, min_samples_split=2, 
                                       min_samples_leaf=1,learning_rate=0.1, max_features='auto', random_state=0)
#subsample : The fraction of samples to be used for fitting the individual base learners. 
#max_features : int, float, 'auto', 'sqrt', 'log2', None

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
estimator.feature_importances_ #The higher, the more important the feature #BaggingRegressor doesn't have

In [ ]:
####################xgboost  parameters ###############################################
silent [default=0]: #print running messages #1 silent
eta [default=0.3] #Analogous to learning rate , Makes the model more robust by shrinking the weights on each step
#Typical final values to be used: 0.01-0.2
min_child_weight [default=1] #the minimum sum of weights of all observations required in a child
#Used to control over-fitting. Higher values prevent a model from learning relations which might be 
#highly specific to the particular sample selected for a tree.
#Too high values can lead to under-fitting 
max_depth [default=6] #maximum depth of a tree #Typical values: 3-10
max_leaf_nodes #maximum number of terminal nodes or leaves in a tree
gamma [default=0] #A node is split only when the resulting split gives a positive reduction 
#in the loss function. Gamma specifies the minimum loss reduction required to make a split
subsample [default=1] #the fraction of observations to be randomly samples for each tree.
colsample_bytree [default=1]#the fraction of columns to be randomly samples for each tree.#Typical values: 0.5-1
#########################################################################################
#########################################################################################
from xgboost import XGBClassifier
estimator = XGBClassifier(objective='binary:logistic', n_estimators=100, learning_rate=0.1, subsample=1,max_depth=3,
                         min_child_weight=1, colsample_bytree=0.8, base_score=0.005, seed =0)
#subsample : Subsample ratio of the training instance.
#colsample_bytree : Subsample ratio of columns when constructing each tree
#base_score: The initial prediction score of all instances, global bias
estimator.fit(X_train, y_train)
y_prob=estimator.predict_proba(X_test)
y_pred=estimator.predict(X_test) 
accuracy=estimator.score(X_train, y_train)
estimator.feature_importances_ #The higher, the more important the feature #BaggingRegressor doesn't have
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from xgboost import XGBRegressor
estimator = XGBRegressor(objective='reg:linear', n_estimators=100, learning_rate=0.1, subsample=1,max_depth=3,
                         min_child_weight=1, colsample_bytree=0.8, base_score=0.005, seed =0)
estimator.fit(X_train, y_train)
y_pred=estimator.predict(X_test) 
R_squared =estimator.score(X_test, y_test)
estimator.feature_importances_ #The higher, the more important the feature #BaggingRegressor doesn't have
sorted(zip(X_train.columns, estimator.feature_importances_ ), key=lambda x:x[1])[::-1]
########################################################################################
from xgboost import DMatrix, cv, train, plot_importance
dtrain = DMatrix(X_train, label=y_train, feature_names=['a','b','c'])
dtest = DMatrix(X_test, feature_names=['a','b','c'])
### parameter tuning
param_grid = [{
        'eta': [0.03],
        'max_depth': [5],
        'min_child_weight': [1],
        'subsample': [1],
        'colsample_bytree': [0.7],
        'objective': ['reg:linear'], #'binary:logistic' for classifier
        'eval_metric': ['rmse'],     # ['error'] for classifier
        'gamma': [0],
        'silent':[1]}]
from sklearn.model_selection import ParameterGrid
best_valid_score=10000000
for params in ParameterGrid(param_grid):
    #print 'max_depth={}, min_child_weight={}'.format(params['max_depth'],params['min_child_weight'])
    #print 'subsample={}, colsample_bytree={}'.format(params['subsample'],params['colsample_bytree'])
    #print 'Gamma={}'.format(params['gamma'])
    #print 'eta={}'.format(params['eta'])
    model = cv(params, dtrain, num_boost_round=2000, nfold=4, early_stopping_rounds=100)
    #num_boost_round : Number of boosting iterations
    #stop, when the score didn't improve for more than early_stopping_rounds
    #print 'The best rmse in the train set :{}'.format(model['train-rmse-mean'].values[-1])
    #print 'The best rmse in the test set :{}'.format(model['test-rmse-mean'].values[-1])
    diff=-model['train-rmse-mean'].values[-1]+model['test-rmse-mean'].values[-1]
    valid_score=model['test-rmse-mean'].values[-1]
    print 'diff = {} | valid_rmse= {}'.format(diff,valid_score)
    if valid_score <= best_valid_score:
        best_valid_score=valid_score
        best_params=params
        print 'Best valid_rmse= {}'.format(valid_score)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
# After finding the best parameter
params = {
        'eta': 0.03,
        'max_depth': 5,
        'min_child_weight': 1,
        'subsample': 1,
        'colsample_bytree': 0.7,
        'objective': 'reg:linear', #'binary:logistic' for classifier
        'eval_metric': 'rmse',     # ['error'] for classifier
        'silent': 1}

model = cv(params, dtrain, num_boost_round=2000, early_stopping_rounds=100,verbose_eval=100, show_stdv=False)
num_boost_rounds = len(model)
print 'Best iteration : {}'.format(num_boost_rounds)
### plot "test-rmse-mean", "train-rmse-mean" vs. iterations
model.loc[30:,["test-rmse-mean", "train-rmse-mean"]].plot()

model = train(dict(params, silent=0), dtrain, num_boost_round= num_boost_rounds)
### plot feature importance
fig, ax = plt.subplots(figsize=(5,5))
plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()
### Prediction
y_pred = model.predict(dtest)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
dtrain = DMatrix(X_train, y_train, feature_names=['a','b','c'])
dvalid = DMatrix(X_valid, y_valid, feature_names=['a','b','c'])
dtest = DMatrix(X_test, feature_names=['a','b','c'])
params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 1.0,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

model = train(params, dtrain, num_boost_round=1000,
                                     evals=[(dtrain, 'train'), (dvalid, 'valid')],
                                     early_stopping_rounds=100,
                                     verbose_eval=100)

num_boost_round =model.best_iteration
print 'Best iteration = {}'.format(model.best_iteration)
print 'validation rmse = {}'.format(model.best_score)
### calculate F scores for all features
features=model.get_score(importance_type='weight') 
A=sorted([(k, features[k]) for k in features], key=lambda x: x[1], reverse=True)
#[('b', 395),
# ('c', 168),
# ('a', 164)]
df_features=DataFrame({'features':zip(*A)[0], 'score': zip(*A)[1]}) 
### Plot feature number vs. importance ratio
import_sum=[]
import_counts=[]
for i in range(2544):#2544 is the largest F score
    importance_sum=np.sum([f[1] for f in A if f[1]>i])
    importance_counts=len([f for f in A if f[1]>i])
    import_sum.append(importance_sum)
    import_counts.append(importance_counts)
import_sum=np.array(import_sum,dtype='float')/import_sum[0]
plt.plot(import_counts, import_sum)
### Prediction
model = train(params, dtrain, num_boost_round=num_boost_round, verbose_eval=100)
y_pred = model.predict(dtest)

### Deep learning models for classification and regression (useful for image recognization or non_linear data)

In [ ]:
#feed forward neural network
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import backend as K
K.set_image_dim_ordering('th')
import h5py
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

# need to Convert labels(Y) to categorical one-hot encoding for multi-class classification
model.add(Dense(10, input_dim=X_train.shape[1], init = 'normal', activation='relu'))
#init='uniform',he_normal,he_uniform
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(5, init = 'normal', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1, init = 'normal', activation='sigmoid')) # for binary classification
#model.add(Dense(3, init='normal', activation = 'softmax')) # for probabilistic multiple-class classification
#model.add(Dense(1, init = 'normal', activation='linear')) # for regression
#activation='softmax','relu','tanh','sigmoid','linear'

#from sklearn.utils import class_weight
#class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
optimizer='adamax'#'adadelta','adam','rmsprop','adagrad', sgd
model.compile(loss = 'binary_crossentropy', optimizer = optimizer,metrics=['accuracy']) #for binary classification
#model.compile(loss='sparse_categorical_crossentropy',optimizer = optimizer,metrics=['accuracy']) #for multi-class classification
#model.compile(loss='categorical_crossentropy',optimizer = optimizer,metrics=['accuracy']) #for multi-class classification
#model.compile(loss = 'mean_squared_error', optimizer = optimizer, metrics=['mse'])  # for regression
#loss = 'categorical_crossentropy','sparse_categorical_crossentropy' for multi-class classification
#if use categorical_crossentropy, when you have 10 classes, the target for each sample should be a 10-dimensional vector 
#loss = 'mean_squared_error', 'mean_absolute_error' 
#loss = 'binary_crossentropy' ,  Also known as logloss. 

#def root_mean_squared_error(y_true, y_pred):
#    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
#model.compile(loss = root_mean_squared_error, optimizer = optimizer) # for regression

#callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')]

testnumber=1
filepath="model_{}.best.hdf5".format(testnumber)
saveBestModel = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
#saveBestModel = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model.fit(X_train, y_train, batch_size=500, nb_epoch=100, 
          validation_data=(X_valid, y_valid),
          #validation_split=0.1,
          #class_weight=class_weight,
          callbacks=[saveBestModel],
          #callbacks=callbacks,
          shuffle=True, verbose=1) 
#train the network for nb_epoch epochs, with batch_size samples per mini-batch.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
###### model evaluation and prediction
model.load_weights(filepath)
#for classification
model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics=['accuracy']) 
scores_valid = model.evaluate(X_valid, y_valid, batch_size=128, verbose=0)
print "{} in the validation set: {}".format(model.metrics_names[0], scores_valid[0])
print "{} in the validation set: {}".format(model.metrics_names[1], scores_valid[1])

y_prob = model.predict_proba(X_test, batch_size=500, verbose=0)
y_pred=np.where(y_prob[:, 0]>i, 1,0)
#~~~~~~~~~~~~
y_pred = model.predict_classes(X_test, batch_size=500, verbose=0)
#~~~~~~~~~~~~
y_pred = model.predict(X_test, batch_size=500, verbose=0)
y_pred = np.where(y_pred>0.5, 1, 0)
#~~~~~~~~~~~~
#for regression
model.compile(loss = 'mean_squared_error', optimizer = optimizer, metrics=['mse'])
scores_valid = model.evaluate(X_valid, y_valid, batch_size=128, verbose=0)
print("MSE in the validation set: %.6f" % (scores_valid))
print("RMSE in the validation set: %.6f" % (np.sqrt(scores_valid)))
y_pred = model.predict(X_test, batch_size=10, verbose=0) 
##########################################################################################
##########################################################################################
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    #model.compile(loss='mean_squared_error', optimizer='adam') for KerasRegressor
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
#model = KerasRegressor(build_fn=create_model, verbose=0)
model.fit(X, Y)

#use grid_search to search for the best batch_size or epochs as well as the model parameters.
optimizers = ['rmsprop', 'adam']
init = ['normal', 'uniform']
epochs = [50, 100, 150]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
##########################################################################################
##########################################################################################
from nolearn.lasagne import NeuralNet
from lasagne.layers import InputLayer, DenseLayer
from lasagne.nonlinearities import softmax, tanh, sigmoid
from lasagne.updates import nesterov_momentum
# three layers: one hidden layer
model = NeuralNet(layers=[('input', InputLayer),('hidden', DenseLayer),('output', DenseLayer)],
                 # layer parameters:
                 input_shape=(None, X_train.shape[1]), 
                 hidden_num_units=10,  # number of units in hidden layer
                 output_nonlinearity=softmax,  # output layer uses identity function
                 output_num_units=2, #number of possible outputs (for binary classification this would be 2) 
                 # optimization method:
                 update=nesterov_momentum,
                 update_learning_rate=0.01, 
                 update_momentum=0.9, 

                 regression=False,  # If you're doing classification you want this off
                 max_epochs=400,  # more epochs can be good, 
                 verbose=1, # enabled so that you see meaningful output when the program runs
                )
model.fit(X_train, y_train)
model.predict(X_test)

In [ ]:
#ex: X_train shape =(1000, 3, 32, 32) for 1000 images
#convolutional neural network
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D 
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras import optimizers
from keras.optimizers import RMSprop, SGD, RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
K.set_image_dim_ordering('th')
import h5py
# for color image classification
model=Sequential()
model.add(Convolution2D(nb_filter=32, nb_row=3, nb_col=3, border_mode='same',input_shape=(3,32,32), activation='relu'))
#Each convolutional layer has a filter size of 3 x 3 and a rectifier activation function
model.add(Convolution2D(nb_filter=32, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
#a max pool layer with a window size of 2×2.

model.add(Convolution2D(nb_filter=16, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
model.add(Convolution2D(nb_filter=16, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(16, init='he_normal', activation = 'tanh'))
model.add(Dropout(0.2))
model.add(Dense(3, init='he_normal', activation = 'softmax')) # for 3-class probability classification
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adamax',#optimizer='adadelta', 'adamax','rmsprop','adam',sgd
              metrics=['accuracy'])
#print(model.summary())
datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,
                             zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')  
##apply image augmentations to artificially increase the size of the training set with new transformed images. 
##A number of random transformations are applied to the initial data by zooming, rotating, shifting or shearing 
##images to prevent overfitting and improving the model generalization error

testnumber=1
filepath="weights32_{}.best.hdf5".format(testnumber)
saveBestModel = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')  
     
model.fit_generator(datagen.flow(X_train, y_train, batch_size=500, shuffle=True), 
                    nb_epoch=nb_epoch, samples_per_epoch=len(X_train), verbose=1, 
                    validation_data=(X_valid, y_valid),
                    callbacks=[saveBestModel])
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model.load_weights(filepath)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adamax',metrics=['accuracy'])
###### model evaluation
scores_valid = model.evaluate(X_valid, y_valid, batch_size=128, verbose=0)
print "{} in the validation set: {}".format(model.metrics_names[0], scores_valid[0])
print "{} in the validation set: {}".format(model.metrics_names[1], scores_valid[1]) 
#print("Best %s: %.6f" % (model.metrics_names[0], scores_valid[0])) # loss
#print("Best %s: %.2f%%" % (model.metrics_names[1], scores_valid[1]*100))  # accuracy  
###### predication
y_pred = model.predict_proba(x_test)
##########################################################################################
##########################################################################################
from lasagne import layers
from lasagne.updates import nesterov_momentum
from lasagne.nonlinearities import softmax, tanh, sigmoid, rectify
from nolearn.lasagne import NeuralNet

# make sure data is in right format, & properly label-encoded
SIZE=32
X_train = X_train.reshape(-1, 1, SIZE, SIZE)
X_test = X_test.reshape(-1, 1, SIZE, SIZE)

model = NeuralNet(layers=[('input', layers.InputLayer),
                          ('conv1', layers.Conv2DLayer),
                          ('pool1', layers.MaxPool2DLayer),
                          ('conv2', layers.Conv2DLayer),
                          ('pool2', layers.MaxPool2DLayer),
                          ('conv3', layers.Conv2DLayer),
                          ('pool3', layers.MaxPool2DLayer),
                          ('hidden4', layers.DenseLayer),
                          ('hidden5', layers.DenseLayer),
                          ('output', layers.DenseLayer)],
                  input_shape=(None, 1, SIZE, SIZE),
                  conv1_num_filters=32, conv1_filter_size=(3, 3), conv1_nonlinearity=rectify, pool1_pool_size=(2, 2), 
                  conv2_num_filters=64, conv2_filter_size=(3, 3), conv2_nonlinearity=rectify, pool2_pool_size=(2, 2), 
                  conv3_num_filters=128, conv3_filter_size=(3, 3), conv3_nonlinearity=rectify, pool3_pool_size=(2, 2), 
                  hidden4_num_units=1000,
                  hidden5_num_units=1000, 
                  output_num_units=2, #number of possible outputs (for binary classification this would be 2) 
                  output_nonlinearity=softmax,
                  
                  update_learning_rate=0.01,
                  update_momentum=0.9,

                  regression=False,
                  max_epochs=20,
                  verbose=1,
                 )
model.fit(X_train, Y_train)
model.predict(X_test)

### Use theano to build a linear regression

In [ ]:
import theano
from theano import tensor as T
from theano.tensor import fscalar
from theano.tensor import fmatrix
from theano.tensor import fvector
from theano import shared
from theano import function

X_train = np.asarray([[0.0], [1.0], [2.0], [3.0], [4.0],
                      [5.0], [6.0], [7.0], [8.0], [9.0]], dtype=np.float32)

y_train = np.asarray([1.0, 1.3, 3.1, 2.0, 5.0, 6.3, 6.6, 7.4, 8.0, 9.0],dtype=np.float32)

eta=0.001
epochs=10
# Initialize arrays
eta0 = fscalar('eta0')
y = fvector(name='y')
X = fmatrix(name='X')
w = theano.shared(np.zeros(shape=(X_train.shape[1] + 1), dtype=np.float32), name='w')

# calculate cost function
net_input = T.dot(X, w[1:]) + w[0]
errors = y - net_input
cost = T.sum(T.pow(errors, 2))
# perform gradient update
gradient = T.grad(cost, wrt=w)
update = [(w, w - eta0 * gradient)]
# compile model
train = function(inputs=[eta0],
                 outputs=cost,
                 updates=update,
                 allow_input_downcast=True,
                 givens={X: X_train,y: y_train})

costs=[]
for _ in range(epochs):
           costs.append(train(eta))

### Clustering models

In [ ]:
from sklearn.mixture import GaussianMixture
estimator=GaussianMixture(n_components=2, covariance_type='tied', n_init=5, init_params='kmeans', random_state=0)
#n_components:The number of mixture components.
#covariance_type:'full' (each component has its own general covariance matrix),
#'tied' (all components share the same general covariance matrix),
#'diag' (each component has its own diagonal covariance matrix),
#'spherical' (each component has its own single variance).
#n_init : The number of initializations to perform. The best results are kept.
#init_params : {‘kmeans’, ‘random’}, defaults to ‘kmeans’. used to initialize the weights
estimator.fit(X_train)
y_pred=estimator.predict(X_test)
y_pred=estimator.predict_proba(X_test)
estimator.means_ # show all cluster centers, shape=(cluster num, feature num)
###########################################################################################
from sklearn.cluster import KMeans
estimator=KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300, random_state=0, n_jobs=-1)
#init :‘k-means++’, ‘random’ 
#n_init :Number of time the k-means algorithm will be run with different centroid seeds.
#max_iter : Maximum number of iterations of the k-means algorithm for a single run.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.cluster import MiniBatchKMeans
#use the KMeans algorithm by Mini-batches which are subsets of the input data, randomly sampled in each training iteration.  
estimator=MiniBatchKMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300, batch_size=100, random_state=0)
#batch_size : Size of the mini batches

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train)
estimator.fit_transform(X_train) #Transform data to a cluster-distance space.
#array([[  1.39754249,  10.67707825],
#       [  1.39754249,  10.67707825],
#       [  1.39754249,  10.67707825],
#shape=(samplenumber, clusternumber), each value is the distance between a sample and a cluster centroid
y_pred=estimator.predict(X_test)
estimator.cluster_centers_  # show all cluster centers, shape=(cluster num, feature num)
estimator.labels_ #Labels of each point
estimator.inertia_ #Sum of distances of samples to their closest cluster center. It is SSE value
###########################################################################################
from sklearn.cluster import AgglomerativeClustering  
estimator=AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='complete')
#linkage : {“ward”, “complete”, “average”}, 
#ward minimizes the variance of the clusters being merged.
#average uses the average of the distances of each observation of the two sets.
#complete uses the maximum distances between all observations of the two sets.
estimator.fit(X_train)
y_pred=estimator.fit_predict(X_train) #Performs clustering on X and returns cluster labels
estimator.labels_ #Labels of each point
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
### Agglomerative clustering using the complete linkage algorim
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage 
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import maxdists
from scipy.cluster.hierarchy import fcluster

X = np.array([[0,0], [3,4], [60,80], [66,88]]) 
pdst=pdist(X, 'euclidean')  #array([   5.,  100.,  110.,   95.,  105.,   10.])
s=squareform(pdst)
#array([[   0.,    5.,  100.,  110.],
#       [   5.,    0.,   95.,  105.],
#       [ 100.,   95.,    0.,   10.],
#       [ 110.,  105.,   10.,    0.]])
####find similar samples 
sample0_similar=s[0].argsort() #array([0, 1, 2, 3], dtype=int64)
X[sample0_similar[:3]] # find 3 most similar samples for sample0
#array([[ 0,  0],
#       [ 3,  4],
#       [60, 80]])

lin=linkage (pdst, method='complete') #method='ward' ward hierarchical clustering
###### Draw dendrogram
plt.figure(figsize=(5, 5))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(lin,
           leaf_rotation=90.,  # rotates the x axis labels
           leaf_font_size=12.,  # font size for the x axis labels
           orientation='left')
plt.show()

###### show cluster labels for each data points
maxdists(lin) #array([   5.,   10.,  110.]) #Returns the maximum distance between any non-singleton cluster.
t=np.median(maxdists(lin)) #t is The threshold to apply when forming flat clusters.
fcluster(lin,t,'distance') #array([1, 1, 2, 2])
#'distance' : points in each cluster have < t.
###########################################################################################
from sklearn.cluster import DBSCAN #Density-based spatial clustering of application with noise
estimator=DBSCAN(eps=0.5, min_samples=5, metric='euclidean', n_jobs=-1)
#eps : The maximum distance between two samples for them to be considered as in the same neighborhood.
#min_samples :The number of samples in a neighborhood for a point to be considered as a core point. 
#metric : The metric to use when calculating distance between instances in a feature array
estimator.fit(X_train)
y_pred=estimator.fit_predict(X_train) #Performs clustering on X and returns cluster labels
estimator.labels_ #Labels of each point #Noisy samples are given the label -1.
###########################################################################################
from sklearn.cluster import AffinityPropagation  #cluster_centers_indices_
#finds instances that are the most representative of others
estimator=AffinityPropagation(max_iter=200)
#max_iter : Maximum number of iterations.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.cluster import MeanShift
# see https://spin.atomicobject.com/2015/05/26/mean-shift-clustering/

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
estimator.fit(X_train)
y_pred=estimator.predict(X_test)
estimator.labels_ #Labels of each point #Noisy samples are given the label -1.
estimator.cluster_centers_ # show all cluster centers, shape=(cluster num, feature num)

### Save and Load estimator and data

In [ ]:
model = LogisticRegression()
model.fit(X_train, Y_train)
# save the model to disk

from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(model, 'filename.pkl') 

# and later you can load it
model = joblib.load('filename.pkl')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import pickle
# now you can save it to a file
with open('filename.pkl', 'wb') as f:
    pickle.dump(model, f)

# and later you can load it
with open('filename.pkl', 'rb') as f:
    model = pickle.load(f)
##########################################################
##########################################################
data=np.array([1])

from sklearn.externals import joblib
# save data
joblib.dump(data,'a.pkl')
# load data 
data=joblib.load('a.pkl', mmap_mode='c')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# save data
from scipy import save
save('filename.npy', data)
# load data 
from scipy import load
load('filename.npy')
##########################################################
##########################################################
# save SQLite database
from sqlite3 import connect 
import os
if os.path.exists('test.sqlite'): 
    os.remove('test.sqlite')
co=connect('test.sqlite')
curs=co.cursor()
curs.execute("CREATE TABLE employees (firstname varchar(32), lastname varchar(32), title varchar(32));")
curs.execute("INSERT INTO employees VALUES('Kelly', 'Koe', 'Engineer');")
co.commit()
co.close()

# load SQLite database
from sqlite3 import connect 
co=connect('test.sqlite')
curs=co.cursor()
curs.execute("SELECT lastname FROM employees;")
for (name) in curs.fetchall():
    print name
co.close()
#(u'Koe',)

### Metrics

In [ ]:
#for classification
y_pred = [0, 1, 1, 0]
y_true = [0, 1, 1, 1]

from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred) #0.75

from sklearn.metrics import precision_score
precision_score(y_true, y_pred) 

from sklearn.metrics import recall_score
recall_score(y_true, y_pred) 

from sklearn.metrics import f1_score
f1_score(y_true, y_pred) 
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Plot f1_score vs. threshold
y_prob=estimator.predict_proba(X_test)
threshold=np.linspace(0.1, 1.0, 100)
f1_list=[]
for i in threshold:
    y_pred=np.where(y_prob>i, 1,0)
    f1_list.append(f1_score(y_true, y_pred))
plt.plot(threshold, f1_list)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

from sklearn.metrics import classification_report
print classification_report(y_true, y_pred) 
#            precision    recall  f1-score   support
#
#          0       0.50      1.00      0.67         1
#          1       1.00      0.67      0.80         3
#
#avg / total       0.88      0.75      0.77         4

from sklearn.metrics import matthews_corrcoef  #-1 -- 1 (perfect)
matthews_corrcoef(y_true, y_pred)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true, y_pred)
#array([[1, 0, 0],
#       [0, 1, 0],
#       [0, 1, 1]])
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()
classes=[0,1,2]
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)
fmt = '.2f' 
thresh = cm.max() / 2.
import itertools
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
    horizontalalignment="center",
    color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
#!!!!!!!!!!!!!!!!!!!!!!!!
import itertools
from sklearn.metrics import confusion_matrix
c=confusion_matrix(y_true, y_pred, labels=[1, 0])
np.set_printoptions(precision=5)
c=c.astype('float')/c.sum()

plt.figure()
plt.imshow(c, interpolation='nearest',cmap=plt.cm.Blues)
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Up', 'Down'])
plt.yticks(tick_marks, ['Up', 'Down'])
plt.colorbar()
thresh = c.max() / 2.

for i, j in itertools.product(range(c.shape[0]), range(c.shape[1])):
    plt.text(j, i, '%.2f' % c[i, j],horizontalalignment="center",
             color="white" if c[i, j] > thresh else "black")
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix in the validation set')
#############################################################################
#############################################################################
y_true = [0, 1, 1, 1]
y_prob = [0.3,0.7,0.8,0.9]
#y_prob= estimator.predict_proba(X_test)[:,1]
from sklearn.metrics import precision_recall_curve#x axis is recall, y axis is precision
precision, recall, thresholds = precision_recall_curve(y_true, y_probability_pred)
zip(precision, recall, thresholds)
#[(1.0, 1.0, 0.34999999999999998),
# (1.0, 0.66666666666666663, 0.40000000000000002),
# (1.0, 0.33333333333333331, 0.80000000000000004)]
from sklearn.metrics import auc
print auc(precision, recall)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_prob)

plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.metrics import roc_curve #Receiver Operating Characteristic
fpr, tpr, thresholds = roc_curve(y_true, y_probability_pred, pos_label=1)
zip(fpr, tpr, thresholds)
#[(0.0, 0.5, 0.80000000000000004),
# (0.5, 0.5, 0.40000000000000002),
# (0.5, 1.0, 0.34999999999999998),
# (1.0, 1.0, 0.10000000000000001)]
from sklearn.metrics import auc 
roc_auc = auc(fpr, tpr) #0.75

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_prob) #0.75

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.metrics import log_loss
#-log P(yt|yp) = -(yt log(yp) + (1 - yt) log(1 - yp))
log_loss(y_true,y_prob)
#############################################################################
#############################################################################
y_pred = [1, 2, 3, 4]
y_true = [2, 2, 3, 4]
#for multi-label classification
from sklearn.metrics import hamming_loss # 1 --- 0 (perfect)
hamming_loss(y_true, y_pred)

from sklearn.metrics import jaccard_similarity_score # larger is better
jaccard_similarity_score(y_true, y_pred)
#############################################################################
#############################################################################
Y = [1,0,0,1,0,1,0,0,1,0]
X = [[10,0, 5], 
     [2, 1, 10], 
     [3, 1, 12], 
     [2, 1, 10], 
     [3, 1, 12], 
     [2, 1, 10], 
     [3, 1, 12], 
     [2, 1, 10], 
     [3, 1, 12], 
     [10,0, 20]]
from sklearn.svm import SVC
classifier = SVC(random_state=0,C=1)

from sklearn.learning_curve import learning_curve#x axis is the number  of train dataset, y axis is accuracy
train_sizes, train_scores, test_scores = learning_curve(classifier, 
                                                        X, Y, cv=2, n_jobs=-1, train_sizes=np.linspace(0.1,1,10), verbose=0)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.figure()
# Plot the average training and test score lines at each training set size
plt.plot(train_sizes, train_scores_mean, 'o-', color="b", label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="r", label="Test score")

# Plot the std deviation as a transparent range at each training set size
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="b")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="r")

# Draw the plot and reset the y-axis
plt.legend(loc="best")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.ylim((0.0, 1.01))
#plt.gca().invert_yaxis()
plt.grid()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from sklearn.svm import SVC
classifier = SVC(random_state=0)
param_range=[0.01,0.1,1, 10, 100]
from sklearn.learning_curve import validation_curve#x axis is the tuning parameter, y axis is accuracy
train_scores, test_scores = validation_curve(classifier, 
                                             X, Y, param_name="C", param_range=param_range,
                                             cv=3, scoring="accuracy", n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.title("Validation Curve")
plt.xlabel("Model Parameter")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

### Image modeling

In [ ]:
###############  image quatization (convert 255 color levels into 64 color levles)  ####
from mahotas import imread
image=imread('test.jpg') / 255
width, height, depth = tuple(image.shape)
image= image.reshape((image.shape[0] * image.shape[1], 3))
print image.shape #(12780288L, 3L)
# Use K-Means to create 64 clusters from 1000 sample colors.
i=np.random.permutation(image.shape[0])
image_train=image[i][:1000]
# cluster the pixel intensities
from sklearn.cluster import KMeans
estimator = KMeans(n_clusters = 64, random_state=0)
estimator.fit(image_train)
y_pred=estimator.predict(image)
compressed_colors=estimator.cluster_centers_ # shape =(64,3)
compressed_image=np.zeros((width, height, depth))
label_idx=0
for i in range(width):
    for j in range(height):
        compressed_image[i][j] = compressed_colors[y_pred[label_idx]]
        label_idx +=1
plt.imshow(compressed_image)
plt.axis('off')
####################################################################################
###############   identify dog or cat images   #####################################
import mahotas as mh
from mahotas.features import surf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.cluster import MiniBatchKMeans

import glob
#Load the images
all_instance_filenames = []
all_instance_targets = []
for f in glob.glob('train/*.jpg'):
    target = 1 if 'cat' in f else 0
    all_instance_filenames.append(f)
    all_instance_targets.append(target)
#Convert the images to grayscale, and extract the SURF descriptors
surf_features = []
counter = 0
for f in all_instance_filenames:
    image = mh.imread(f, as_grey=True)
    surf_features.append(surf.surf(image, decriptor_only=True))

#Split the images into training and testing data
train_len = int(len(all_instance_filenames)*.80)
X_train_surf_features = np.concatenate(surf_features[:train_len])
X_test_surf_features = np.concatenate(surf_features[train_len:])
y_train = all_instance_targets[:train_len]
y_test = all_instance_targets[train_len:]

#Group the extracted descriptors into 300 clusters. Use MiniBatchKMeans
#to compute the distances to the centroids for a sample of the instances.
n_clusters = 300
estimator = MiniBatchKMeans(n_clusters=n_clusters)
estimator.fit_transform(X_train_surf_features)

#Construct feature vectors for training and testing data. Find the cluster
#associated with each of the extracted SURF descriptors, and count them using np.bincount()
X_train = []
for instance in surf_features[:train_len]:
    clusters = estimator.predict(instance)
    features = np.bincount(clusters)
    if len(features) < n_clusters:
        features = np.append(features, np.zeros((1, n_clusters-len(features))))
    X_train.append(features)

X_test = []
for instance in surf_features[train_len:]:
    clusters = estimator.predict(instance)
    features = np.bincount(clusters)
    if len(features) < n_clusters:
        features = np.append(features, np.zeros((1, n_clusters-len(features))))
    X_test.append(features)
    
#Train a logistic regression classifier on the feature vectors and targets,
clf = LogisticRegression(C=0.001, penalty='l2')
clf.fit_transform(X_train, y_train)
predictions = clf.predict(X_test)
print 'Accuracy: ', accuracy_score(y_test, predictions)

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
image_surf=[]
for f in df['filename']:
    image=mh.imread(f,as_grey=True)
    s=surf.surf(image, 4, 6, 1, descriptor_only=True)
    image_surf.append(np.array(s))
image_surf_combined=np.concatenate(image_surf)

from sklearn.cluster import KMeans
distortions = [] #Sum of distances of samples to their closest cluster center#within_cluster SSE
for i in range(400, 850, 50):
    km=KMeans(n_clusters=i,init='k-means++',random_state=0, n_jobs=-1)
    km.fit(image_surf_combined)
    distortions.append(km.inertia_)
plt.plot(range(50,450,50), distortions)
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()

### Music genre classification

In [ ]:
from scipy.io.wavfile import read
from scipy import fft
sample_rate, x =read('xx.wav')#sample_rate unit: Hz
####### plot specgram
plt.specgram(x, Fs=sample_rate, xextent=(0,30))#plot first 30 sec 
####### use Fast Fourier transform frequency as features
fft_features=abs(fft(x)[:1000])#choose first 10000 fft components

### Language modeling

In [ ]:
## predict what is the next letter 
text='SUPPOSING that Truth is a woman what then  Is'
# unique words
chars = sorted(list(set(text))) 
#[' ', 'G', 'I', 'N', 'O', 'P', 'S', 'T', 'U', 'a', 'e', 'h', 'i', 'm', 'n', 'o', 'r', 's', 't', 'u', 'w']
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 5
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences)) #('nb sequences:', 14)
zip(sentences, next_chars)
#[('SUPPO', 'S'),
# ('POSIN', 'G'),
# ('ING t', 'h'),
# (' that', ' '),
# ('at Tr', 'u'),
# ('Truth', ' '),
# ('th is', ' '),
# ('is a ', 'w'),
# ('a wom', 'a'),
# ('oman ', 'w'),
# ('n wha', 't'),
# ('hat t', 'h'),
# (' then', ' '),
# ('en  I', 's')]

## create X, y data
X = np.zeros((len(sentences), maxlen, len(chars)))
print X.shape #(14L, 5L, 21L)
y = np.zeros((len(sentences), len(chars)))
print y.shape #(14L, 21L)
char_indices = dict((c, i) for i, c in enumerate(chars))
#{'a': 9, ' ': 0, 'e': 10, 'i': 12, 'G': 1, 'o': 15, 'I': 2, 'h': 11, 'm': 13, 
# 's': 17, 'O': 4, 'N': 3, 'P': 5, 'S': 6, 'r': 16, 'U': 8, 'T': 7, 'w': 20, 'u': 19, 'n': 14, 't': 18}
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
## create LSTM model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(128, input_shape=(5, 21)))
model.add(Dense(21))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.fit(X, y, batch_size=3, nb_epoch=1)

## model prediction
import random
start_index = random.randint(0, len(text) - maxlen - 1)
sentence = text[start_index: start_index + maxlen]
print sentence #POSIN
X_test = np.zeros((1, maxlen, len(chars)))
for t, char in enumerate(sentence):
    X_test[0, t, char_indices[char]] = 1.
preds = model.predict(X_test, verbose=0)[0]
#array([ 0.0512807 ,  0.07149747,  0.04306439,  0.04000025,  0.04201484,
#        0.03914934,  0.06487428,  0.04007942,  0.03958567,  0.04070712,
#        0.03861614,  0.05520309,  0.04078073,  0.04078181,  0.04187801,
#        0.04210004,  0.03678251,  0.04980318,  0.05945305,  0.06003801,
#        0.06230993], dtype=float32)
probas=np.random.multinomial(1, preds, 1)
#array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
next_index = np.argmax(probas)  #1
indices_char = dict((i, c) for i, c in enumerate(chars))
next_char = indices_char[next_index]
print next_char

In [ ]:
## Classify Movie reviews as good or bad
from keras.datasets import imdb
max_features = 20000
maxlen = 100  # cut texts after this number of words (among top max_features most common words)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
#X_train has 25000 reviews. each review has a lot of words represented by numbers
#y_train is array([1, 0, 0, ..., 0, 1, 0]), 25000 elements. Good review is 1. 
from keras.preprocessing import sequence
maxlen = 100  # for each review, choose the first 100 common words
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape) #('X_train shape:', (25000, 100))
print('X_test shape:', X_test.shape)   #('X_test shape:', (25000, 100))

## train data
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# choose one of three layers
model.add(GRU(128)) 
model.add(SimpleRNN(128)) 
model.add(LSTM(128))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X_train, y_train, batch_size=32, epochs=1, validation_data=(X_test, y_test))

In [ ]:
# from words 1, 2, 3, ... , (n-1) to predict words 2, 3, 4, ... , (n) in a sentence
X=[('A very clean and well decorated empty bathroom'),
  ('This is very bad house')]
from keras.preprocessing.text import Tokenizer
vocabularySize = 100  # vocabulary size.
# Split sentences into words, and define a vocabulary with the most common words.
tokenizer = Tokenizer(nb_words = vocabularySize, filters = '!"#$%&()*+,-./:;<=>?@\\^_`{|}~\t\n') 
tokenizer.fit_on_texts(X)

# Convert the sentences into sequences of word ids using our vocabulary.
X_Sequences = tokenizer.texts_to_sequences(X) #[[2, 1, 3, 4, 5, 6, 7, 8], [9, 10, 1, 11, 12]]
# Keep dictionaries that map ids -> words, and words -> ids.
word2id = tokenizer.word_index
#{'a': 2,
# 'and': 4,
# 'bad': 11,
# 'bathroom': 8,
# 'clean': 3,
# 'decorated': 6,
# 'empty': 7,
# 'house': 12,
# 'is': 10,
# 'this': 9,
# 'very': 1,
# 'well': 5}
id2word = {idx: word for (word, idx) in word2id.items()}
#{1: 'very',
# 2: 'a',
# 3: 'clean',
# 4: 'and',
# 5: 'well',
# 6: 'decorated',
# 7: 'empty',
# 8: 'bathroom',
# 9: 'this',
# 10: 'is',
# 11: 'bad',
# 12: 'house'}
maxSequenceLength = max([len(seq) for seq in X_Sequences])  # 8 Find the sentence with most words.

from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(X_Sequences, maxlen = (maxSequenceLength + 1), padding = 'post', truncating = 'post')
id2word[0] = 'END'
word2id['END'] = 0
#data is array([[ 2,  1,  3,  4,  5,  6,  7,  8,  0],
#                [ 9, 10,  1, 11, 12,  0,  0,  0,  0]], dtype=int32)

inputData = data[:, :-1]  # for training model
print inputData.shape #(2, 8)
outputData = data[:, 1:]  # for target variables
print outputData.shape #(2, 8)
outputLabels = np.expand_dims(outputData, -1)
print outputLabels.shape  #(2, 8, 1)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from keras.layers import Input, Dense, Activation, SimpleRNN
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

words = Input(batch_shape=(None, maxSequenceLength), name = "input")
# Build a matrix of size vocabularySize x 300 where each row corresponds to a "word embedding" vector.
# This layer will convert replace each word-id with a word-vector of size 300.
embeddings = Embedding(vocabularySize, 300, name = "embeddings")(words)
# Pass the word-vectors to the LSTM layer.
hiddenStates = SimpleRNN(512, return_sequences = True, input_shape=(maxSequenceLength, 300), name = "rnn")(embeddings)
# Apply a linear (Dense) layer to the outputs of the LSTM at each time step.
denseOutput = TimeDistributed(Dense(vocabularySize), name = "linear")(hiddenStates)
predictions = TimeDistributed(Activation("softmax"), name = "softmax")(denseOutput)                                      
# Build the computational graph by specifying the input, and output of the network.
model = Model(input = words, output = predictions)

model.compile(loss='sparse_categorical_crossentropy', optimizer = RMSprop(lr = 0.001))
print(model.summary())
#_________________________________________________________________
#Layer (type)                 Output Shape              Param #   
#=================================================================
#input (InputLayer)           (None, 8)                 0         
#_________________________________________________________________
#embeddings (Embedding)       (None, 8, 300)            30000     
#_________________________________________________________________
#rnn (SimpleRNN)              (None, 8, 512)            416256    
#_________________________________________________________________
#linear (TimeDistributed)     (None, 8, 100)            51300     
#_________________________________________________________________
#softmax (TimeDistributed)    (None, 8, 100)            0    

model.fit(inputData, outputLabels, batch_size = 1, nb_epoch = 2)

y_pred=model.predict(data[:,:-1])
print y_pred.shape #(2, 8, 100)